In [1]:
### Import modules
from pandas import Series, DataFrame
import pandas as pd
import xport  ## write sas datasets to local drive
from sas7bdat import SAS7BDAT  ## read SAS datasets from local drive

## Read ADaM datasets

In [2]:
### Read ADAE SAS datasets
with open('../adam/adae.xpt', 'rb') as _adae1: 
    _adae2 = xport.to_columns(_adae1)
    _adae3 = pd.DataFrame(_adae2)

In [3]:
### Read ADSL SAS datasets
with open('../adam/adsl.xpt', 'rb') as _adsl1: 
    _adsl2 = xport.to_columns(_adsl1)
    _adsl3 = pd.DataFrame(_adsl2)

## Read SDTM dataset

In [4]:
### Read SDTM EG datasets
with open('../sdtm/eg.xpt', 'rb') as _eg1: 
    _eg2 = xport.to_columns(_eg1)
    _eg3 = pd.DataFrame(_eg2)

In [5]:
### Read SDTM EX datasets
with open('../sdtm/ex.xpt', 'rb') as _ex1: 
    _ex2 = xport.to_columns(_ex1)
    _ex3 = pd.DataFrame(_ex2)

## Prepare ADSL dataset

In [6]:
_adsl = _adsl3[['USUBJID','WEIGHTB']]

## Prepare EG dataset

In [7]:
## Keeping only AORTIC VALVE REGURGITATION & MITRAL VALVE REGURGITATION
_eg4 = _eg3[(_eg3.EGTEST == 'MITRAL VALVE REGURGITATION') | (_eg3.EGTEST == 'AORTIC VALVE REGURGITATION')]
_eg = _eg4[['USUBJID','EGTEST','EGSTRESC','EGDTC','VISIT']] ## keeping certain variables
print(_eg.EGTEST.value_counts()) ## frequency of egtest
_eg.head()


MITRAL VALVE REGURGITATION    1235
AORTIC VALVE REGURGITATION    1235
Name: EGTEST, dtype: int64


,USUBJID,EGTEST,EGSTRESC,EGDTC,VISIT
163,ZX008-1501-0101-01,AORTIC VALVE REGURGITATION,ABSENT,2016-09-14,OLE Visit 1 - Day 1
164,ZX008-1501-0101-01,AORTIC VALVE REGURGITATION,ABSENT,2016-10-27,OLE Visit 4 - Month 2
165,ZX008-1501-0101-01,AORTIC VALVE REGURGITATION,ABSENT,2017-01-04,OLE Visit 5 - Month 3
166,ZX008-1501-0101-01,AORTIC VALVE REGURGITATION,ABSENT,2017-03-28,OLE Visit 6 - Month 6
167,ZX008-1501-0101-01,AORTIC VALVE REGURGITATION,ABSENT,2017-06-23,OLE Visit 7 - Month 9


## Get ADEC and  Prepare EX dataset

## Prepare SUPPEC dataset

In [99]:
### Read SDTM SUPPEC dataset
with open('../sdtm/suppec.xpt', 'rb') as _suppec1: 
    _suppec2 = xport.to_columns(_suppec1)
    _suppec3 = pd.DataFrame(_suppec2)

In [106]:
_suppec4 = _suppec3[_suppec3.QNAM == 'TREATVT']
_suppec4['QVAL'].value_counts()

ZX008 0.8 mg/kg/day    42249
ZX008 0.4 mg/kg/day    33664
ZX008 0.2 mg/kg/day    33603
ZX008 0.6 mg/kg/day    31514
ZX008 0.5 mg/kg/day     3316
ZX008 0.3 mg/kg/day       57
Name: QVAL, dtype: int64

In [113]:
## Assign Study Medication and Dose
_suppec3.loc[((_suppec3['QVAL']=='Yes, the full dose') & (_suppec3['QNAM']=='ECTAKMED')), 'QVALN'] = 1
_suppec3.loc[((_suppec3['QVAL']=='Yes, a partial dose') & (_suppec3['QNAM']=='ECTAKMED')), 'QVALN'] = 2
_suppec3.loc[((_suppec3['QVAL']=='No') & (_suppec3['QNAM']=='ECTAKMED')), 'QVALN'] = 0
print('Take medicine?', '\n', _suppec3['QVALN'].value_counts())

_suppec3.loc[((_suppec3['QVAL']=='ZX008 0.8 mg/kg/day') & (_suppec3['QNAM']=='TREATVT')), 'QVALN'] = 0.8
_suppec3.loc[((_suppec3['QVAL']=='ZX008 0.4 mg/kg/day') & (_suppec3['QNAM']=='TREATVT')), 'QVALN'] = 0.4
_suppec3.loc[((_suppec3['QVAL']=='ZX008 0.2 mg/kg/day') & (_suppec3['QNAM']=='TREATVT')), 'QVALN'] = 0.2
_suppec3.loc[((_suppec3['QVAL']=='ZX008 0.6 mg/kg/day') & (_suppec3['QNAM']=='TREATVT')), 'QVALN'] = 0.6
_suppec3.loc[((_suppec3['QVAL']=='ZX008 0.5 mg/kg/day') & (_suppec3['QNAM']=='TREATVT')), 'QVALN'] = 0.5
_suppec3.loc[((_suppec3['QVAL']=='ZX008 0.3 mg/kg/day') & (_suppec3['QNAM']=='TREATVT')), 'QVALN'] = 0.3
print('Take medicine?', '\n', _suppec3['QVALN'].value_counts())

Take medicine? 
 1.0    143151
2.0      1290
0.0       290
Name: QVALN, dtype: int64
Take medicine? 
 1.0    143151
0.8     42249
0.4     33664
0.2     33603
0.6     31514
0.5      3316
2.0      1290
0.0       290
0.3        57
Name: QVALN, dtype: int64


In [115]:
## Transpose data to 
print(len(_suppec3))
_suppec4 = pd.pivot_table(_suppec3, values="QVALN", columns=['QNAM'], index=['USUBJID','IDVARVAL'])
_suppec5 = _suppec4.reset_index()
print(len(_suppec5))
_suppec5.head()

289134
144732


QNAM,USUBJID,IDVARVAL,ECTAKMED,TREATVT
0,ZX008-1501-0101-01,1,1.0,0.2
1,ZX008-1501-0101-01,10,1.0,0.2
2,ZX008-1501-0101-01,100,1.0,0.4
3,ZX008-1501-0101-01,1000,1.0,0.8
4,ZX008-1501-0101-01,1001,1.0,0.8


In [132]:
## Subset ones of no dose (ECTAKMED = 0)
_suppec6 = _suppec5[(_suppec5.ECTAKMED != 0) ]
_suppec6['ECSEQ'] = _suppec6.IDVARVAL.astype('float64')
_suppec = _suppec6.copy()
print(len(_suppec))
_suppec.head()

144442


C:\Users\klee\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


QNAM,USUBJID,IDVARVAL,ECTAKMED,TREATVT,ECSEQ
0,ZX008-1501-0101-01,1,1.0,0.2,1.0
1,ZX008-1501-0101-01,10,1.0,0.2,10.0
2,ZX008-1501-0101-01,100,1.0,0.4,100.0
3,ZX008-1501-0101-01,1000,1.0,0.8,1000.0
4,ZX008-1501-0101-01,1001,1.0,0.8,1001.0


In [130]:
a = _suppec6.loc[1, 'IDVARVAL']
print(type(a), a)
b = _ec3.loc[1, 'ECSEQ']
print(type(b), b)

<class 'str'> 10
<class 'numpy.float64'> 2.0


## Prepare SDTM EC dataset and merge with SUPPEC

In [120]:
### Read SDTM EC dataset
with open('../sdtm/ec.xpt', 'rb') as _ec1: 
    _ec2 = xport.to_columns(_ec1)
    _ec3 = pd.DataFrame(_ec2)


In [137]:
## merge EC with Suppec
_ec4 = pd.merge(_ec3, _suppec, how='left', on=['USUBJID','ECSEQ'] )
_ec4['ECSTDT'] = _ec4.ECSTDTC.str[:10]
print(_ec4[['USUBJID','ECSEQ','ECSTDT','ECSTDTC','ECTAKMED','TREATVT']].head())
#print(_ec4.groupby(['USUBJID','TREATVT'])['ECTAKMED'].count())
print(len(_ec3), len(_ec4))

              USUBJID  ECSEQ      ECSTDT              ECSTDTC  ECTAKMED  \
0  ZX008-1501-0101-01    1.0  2016-09-28  2016-09-28T07:45:00       1.0   
1  ZX008-1501-0101-01    2.0  2016-09-28  2016-09-28T19:45:00       1.0   
2  ZX008-1501-0101-01    3.0  2016-09-29  2016-09-29T07:50:00       1.0   
3  ZX008-1501-0101-01    4.0  2016-09-29  2016-09-29T19:50:00       1.0   
4  ZX008-1501-0101-01    5.0  2016-09-30  2016-09-30T07:50:00       1.0   

   TREATVT  
0      0.2  
1      0.2  
2      0.2  
3      0.2  
4      0.2  
144732 144732


In [ ]:
## Find the first and last observation of USUBJID and TREATVT
_ec_f = _ec4.groupby(['USUBJID','TREATVT']).first()
_ec_l = _ec4.groupby(['USUBJID','TREATVT']).last()

In [175]:
_ec_f['OBS'] = 0
_ec_l['OBS'] = 1
## Concatenate first and last records 
_ec5 = pd.concat([_ec_f, _ec_l])
_ec6 = _ec5.reset_index()

In [176]:
print(len(_ec_f), len(_ec_l))

650 650


In [177]:
## Sort by USUBJID, TREATVT and ECSTDTC
_ec7 = _ec6.sort_values(by=['USUBJID','TREATVT','ECSTDTC'])
_ec8 = _ec7.reset_index()

## copy dataframe
_ec9 = _ec8.copy()

## Set up initial variables
_usubjid = _ec9.loc[0,'USUBJID']
_treatvt = _ec9.loc[0,'TREATVT']
_ecstdtc = _ec9.loc[0,'ECSTDTC']
_ecendtc = _ec9.loc[0,'ECENDTC']

last_index = _ec9.index[-1] ## last index number 

In [212]:
## use for-loop to derive days of exposure
for i, _df in _ec9.iterrows():
    # for the current iteration
    _ecendtc = _df['ECENDTC']
    #Firat and Last observation of each group. 
    if i > 0:
        if _df2['OBS'] == 1:      
            _ec9.loc[i, 'EXSTDTC'] = _ecstdtc[:10]
            _ec9.loc[i, 'EXENDTC'] = _ecendtc[:10]
    # for the next iteration 
    _ecstdtc = _df['ECSTDTC'] # for next start 

In [213]:
## Select the last observation for each USUBJID and TREATVT (Dose)
_ec10 = _ec9[_ec9['OBS'] == 1]
_ec10['EXDOSTXT'] = _ec9['TREATVT'] ## convert to Dose  
print(len(_ec10))
_ec10[['USUBJID','TREATVT','EXDOSTXT','EXSTDTC','EXENDTC','ECSTDTC','ECENDTC','OBS']][:20]

650


C:\Users\klee\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,USUBJID,TREATVT,EXDOSTXT,EXSTDTC,EXENDTC,ECSTDTC,ECENDTC,OBS
1,ZX008-1501-0101-01,0.2,0.2,2016-09-28,2016-10-26,2016-10-26T19:50:00,2016-10-26T19:50:00,1
3,ZX008-1501-0101-01,0.4,0.4,2016-10-27,2016-11-28,2016-11-28T19:50:00,2016-11-28T19:50:00,1
5,ZX008-1501-0101-01,0.6,0.6,2016-11-29,2017-06-22,2017-06-22T19:33:00,2017-06-22T19:33:00,1
7,ZX008-1501-0101-01,0.8,0.8,2017-06-23,2018-06-17,2018-06-17T19:15:00,2018-06-17T19:15:00,1
9,ZX008-1501-0101-02,0.2,0.2,2016-10-05,2016-11-01,2016-11-01T19:40:00,2016-11-01T19:40:00,1
11,ZX008-1501-0101-02,0.6,0.6,2016-11-02,2017-01-03,2017-01-03T19:04:00,2017-01-03T19:04:00,1
13,ZX008-1501-0101-02,0.8,0.8,2017-01-04,2018-06-14,2018-06-14T07:24:00,2018-06-14T07:24:00,1
15,ZX008-1501-0101-04,0.2,0.2,2016-10-05,2016-11-01,2016-11-01T19:04:00,2016-11-01T19:04:00,1
17,ZX008-1501-0101-04,0.4,0.4,2016-11-02,2016-12-06,2016-12-06T19:11:00,2016-12-06T19:11:00,1
19,ZX008-1501-0101-04,0.6,0.6,2016-12-07,2017-01-03,2017-01-03T20:40:00,2017-01-03T20:40:00,1


In [272]:
## select only variables needed
_ex = _ec10[['USUBJID','EXDOSTXT','EXSTDTC','EXENDTC']]
print(len(_ec))
print(_ec[:20])

650
               USUBJID  EXDOSTXT     EXSTDTC     EXENDTC
1   ZX008-1501-0101-01       0.2  2016-09-28  2016-10-26
3   ZX008-1501-0101-01       0.4  2016-10-27  2016-11-28
5   ZX008-1501-0101-01       0.6  2016-11-29  2017-06-22
7   ZX008-1501-0101-01       0.8  2017-06-23  2018-06-17
9   ZX008-1501-0101-02       0.2  2016-10-05  2016-11-01
11  ZX008-1501-0101-02       0.6  2016-11-02  2017-01-03
13  ZX008-1501-0101-02       0.8  2017-01-04  2018-06-14
15  ZX008-1501-0101-04       0.2  2016-10-05  2016-11-01
17  ZX008-1501-0101-04       0.4  2016-11-02  2016-12-06
19  ZX008-1501-0101-04       0.6  2016-12-07  2017-01-03
21  ZX008-1501-0101-04       0.8  2017-01-04  2018-06-16
23  ZX008-1501-0101-06       0.2  2016-09-01  2016-09-29
25  ZX008-1501-0101-06       0.4  2016-09-30  2016-10-25
27  ZX008-1501-0101-06       0.8  2016-10-26  2018-06-14
29  ZX008-1501-0101-07       0.2  2016-11-17  2016-12-11
31  ZX008-1501-0101-07       0.6  2016-12-12  2017-01-09
33  ZX008-1501-0101-07     

## Prepare ADSI datasets
### Merge with ADAE with ADSL

In [191]:
## Prepare AESI with treatment emergent 
_adae4 = _adae3[((_adae3.AESI == 'Y' ) & (_adae3.TRTEMFL == 'Y' ))]
_adae5 = _adae4[['STUDYID','USUBJID','SUBJID','SEX','AGE','AETERM','AEDECOD','AESTDTC','ASTDT','ASDY','AEACN','AESEV','AESER','AEOUT','TRTEMFL']]

In [192]:
## merge with ADSL to obtain baseline weight
adae_sl = pd.merge(_adae5, _adsl, how='left', on='USUBJID')

In [193]:
print(len(_adae5))
print(len(adae_sl))

74
74


In [194]:
adae_sl['AEDECOD'].value_counts()

Echocardiogram abnormal               34
Blood glucose decreased               14
Blood pressure diastolic increased     7
Blood pressure increased               6
Heart rate increased                   3
Hypoglycaemia                          2
Tachycardia                            2
Blood prolactin increased              2
Blood pressure systolic increased      1
Dysphagia                              1
Mitral valve incompetence              1
Cardiac murmur                         1
Name: AEDECOD, dtype: int64

In [195]:
adae_sl.head()

,STUDYID,USUBJID,SUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,ASTDT,ASDY,AEACN,AESEV,AESER,AEOUT,TRTEMFL,WEIGHTB
0,ZX008-1503,ZX008-1501-0101-01,0101-01,F,4.0,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04,20823.0,99.0,DOSE NOT CHANGED,MILD,N,RECOVERED/RESOLVED,Y,17.69
1,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,DOSE NOT CHANGED,MILD,N,RECOVERED/RESOLVED,Y,45.90
2,ZX008-1503,ZX008-1501-0101-06,0101-06,M,6.0,DECREASE IN SERUM BLOOD GLUCOSE LEVEL,Blood glucose decreased,2018-03-01,21244.0,547.0,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,Y,25.40
3,ZX008-1503,ZX008-1501-0101-09,0101-09,F,6.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,21104.0,268.0,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,Y,19.60
4,ZX008-1503,ZX008-1501-0101-14,0101-14,M,12.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-24,21116.0,225.0,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,Y,35.70


In [196]:
print(adae_sl['TRTEMFL'].value_counts())

Y    74
Name: TRTEMFL, dtype: int64


## Table 2

### Get AESI dataset for table 2

In [197]:
## Subset of AEDECOD of Echocardiogram abnormal                                                                             
aesi_t2 = adae_sl[(adae_sl.AEDECOD == "Echocardiogram abnormal") | (adae_sl.AEDECOD == "Mitral valve incompetence") ]

print(len(aesi_t2))
#aesi_t2

35


In [198]:
print(aesi_t2.columns.tolist())

['STUDYID', 'USUBJID', 'SUBJID', 'SEX', 'AGE', 'AETERM', 'AEDECOD', 'AESTDTC', 'ASTDT', 'ASDY', 'AEACN', 'AESEV', 'AESER', 'AEOUT', 'TRTEMFL', 'WEIGHTB']


In [199]:
## Print to local drive
t4 = open('./out/aesi_t2_init.rtf', 'w')
print(aesi_t2[['USUBJID', 'SEX', 'AGE', 'AETERM', 'AEDECOD', 'AESTDTC', 'AESEV', 'AESER', 'AEOUT', 'TRTEMFL', 'WEIGHTB']].to_string(), file=t4)

t4.close()

### Merge with EG

In [200]:
## merge with EG to obtain EG data
aesi_t2_eg1 = pd.merge(aesi_t2, _eg, how='left', on=['USUBJID'])

## Create ECHO value
aesi_t2_eg1['ECHO'] = aesi_t2_eg1.EGSTRESC.str[:1] ## New ECHO variables - extract first string of ECHO reading

## Printing
print("Number of records in AESI of Echocardiogram abnormal : ", len(aesi_t2))
print("Number of records in AESI of Echocardiogram abnormal with ECHO data : ", len(aesi_t2_eg1))
aesi_t2_eg1.head()
#print(aesi_t2_eg1.USUBJID.value_counts())

Number of records in AESI of Echocardiogram abnormal :  35
Number of records in AESI of Echocardiogram abnormal with ECHO data :  402


,STUDYID,USUBJID,SUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,ASTDT,ASDY,...,AESEV,AESER,AEOUT,TRTEMFL,WEIGHTB,EGTEST,EGSTRESC,EGDTC,VISIT,ECHO
0,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.9,AORTIC VALVE REGURGITATION,ABSENT,2016-09-22,OLE Visit 1 - Day 1,A
1,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.9,AORTIC VALVE REGURGITATION,ABSENT,2016-11-09,OLE Visit 4 - Month 2,A
2,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.9,AORTIC VALVE REGURGITATION,ABSENT,2017-01-04,OLE Visit 5 - Month 3,A
3,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.9,AORTIC VALVE REGURGITATION,ABSENT,2017-04-12,OLE Visit 6 - Month 6,A
4,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.9,AORTIC VALVE REGURGITATION,ABSENT,2017-07-11,OLE Visit 7 - Month 9,A


In [201]:
## Keeping only records of EG happens before AE - AESTDTC >= EGDTC
aesi_t2_eg_b1 = aesi_t2_eg1[aesi_t2_eg1.AESTDTC >= aesi_t2_eg1.EGDTC]
print("Number of records in AESI of Echocardiogram abnormal with proper ECHO : ", len(aesi_t2_eg_b1))
#print(aesi_t2_eg2.USUBJID.value_counts())
aesi_t2_eg_b1.head()

Number of records in AESI of Echocardiogram abnormal with proper ECHO :  316


,STUDYID,USUBJID,SUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,ASTDT,ASDY,...,AESEV,AESER,AEOUT,TRTEMFL,WEIGHTB,EGTEST,EGSTRESC,EGDTC,VISIT,ECHO
0,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.9,AORTIC VALVE REGURGITATION,ABSENT,2016-09-22,OLE Visit 1 - Day 1,A
1,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.9,AORTIC VALVE REGURGITATION,ABSENT,2016-11-09,OLE Visit 4 - Month 2,A
2,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.9,AORTIC VALVE REGURGITATION,ABSENT,2017-01-04,OLE Visit 5 - Month 3,A
3,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.9,AORTIC VALVE REGURGITATION,ABSENT,2017-04-12,OLE Visit 6 - Month 6,A
4,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.9,AORTIC VALVE REGURGITATION,ABSENT,2017-07-11,OLE Visit 7 - Month 9,A


In [202]:
## Remove ABSENT results 
aesi_t2_eg_b2 = aesi_t2_eg_b1[aesi_t2_eg_b1.EGSTRESC != 'ABSENT']
print(len(aesi_t2_eg_b2), "\nFreq before: ", aesi_t2_eg_b1['EGSTRESC'].value_counts(), 
      "\nFreq after: ", aesi_t2_eg_b2['EGSTRESC'].value_counts())
aesi_t2_eg_b2

48 
Freq before:  ABSENT    268
TRACE      48
Name: EGSTRESC, dtype: int64 
Freq after:  TRACE    48
Name: EGSTRESC, dtype: int64


,STUDYID,USUBJID,SUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,ASTDT,ASDY,...,AESEV,AESER,AEOUT,TRTEMFL,WEIGHTB,EGTEST,EGSTRESC,EGDTC,VISIT,ECHO
15,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,21187.0,456.0,...,MILD,N,RECOVERED/RESOLVED,Y,45.90,MITRAL VALVE REGURGITATION,TRACE,2018-01-03,OLE Visit 9 - Month 15,T
30,ZX008-1503,ZX008-1501-0101-09,0101-09,F,6.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,21104.0,268.0,...,MILD,N,NOT RECOVERED/NOT RESOLVED,Y,19.60,MITRAL VALVE REGURGITATION,TRACE,2017-10-12,OLE Visit 7 - Month 9,T
43,ZX008-1503,ZX008-1501-0101-14,0101-14,M,12.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-24,21116.0,225.0,...,MILD,N,NOT RECOVERED/NOT RESOLVED,Y,35.70,MITRAL VALVE REGURGITATION,TRACE,2017-10-24,OLE Visit 12 - Month 24,T
54,ZX008-1503,ZX008-1501-0103-17,0103-17,M,10.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-02-07,21222.0,185.0,...,MILD,N,NOT RECOVERED/NOT RESOLVED,Y,36.20,MITRAL VALVE REGURGITATION,TRACE,2018-02-07,OLE Visit 6 - Month 6,T
69,ZX008-1503,ZX008-1501-0104-03,0104-03,F,2.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2018-01-02,21186.0,386.0,...,MILD,N,NOT RECOVERED/NOT RESOLVED,Y,11.52,MITRAL VALVE REGURGITATION,TRACE,2017-12-11,OLE Visit 8 - Month 12,T
78,ZX008-1503,ZX008-1501-0104-07,0104-07,F,6.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-11-13,21136.0,197.0,...,MILD,N,NOT RECOVERED/NOT RESOLVED,Y,20.00,MITRAL VALVE REGURGITATION,TRACE,2017-04-11,OLE Visit 1 - Day 1,T
81,ZX008-1503,ZX008-1501-0104-07,0104-07,F,6.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-11-13,21136.0,197.0,...,MILD,N,NOT RECOVERED/NOT RESOLVED,Y,20.00,MITRAL VALVE REGURGITATION,TRACE,2017-10-30,OLE Visit 6 - Month 6,T
83,ZX008-1503,ZX008-1501-0104-07,0104-07,F,6.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-11-13,21136.0,197.0,...,MILD,N,NOT RECOVERED/NOT RESOLVED,Y,20.00,MITRAL VALVE REGURGITATION,TRACE,2017-04-11,Visit 12 (Clinic),T
90,ZX008-1503,ZX008-1501-0104-09,0104-09,F,7.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-08-31,21062.0,71.0,...,MILD,N,RECOVERED/RESOLVED,Y,27.99,MITRAL VALVE REGURGITATION,TRACE,2017-08-11,OLE Visit 3 - Month 1,T
110,ZX008-1503,ZX008-1501-0107-02,0107-02,M,8.0,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2018-02-22,21237.0,527.0,...,MILD,N,RECOVERING/RESOLVING,Y,49.53,MITRAL VALVE REGURGITATION,TRACE,2018-02-14,OLE Visit 10 - Month 18,T


In [203]:
## Extract Last Records for Before AESTDTC

## sort data
aesi_t2_eg_b3 = aesi_t2_eg_b2.sort_values(by=['USUBJID','AETERM','AEDECOD','AESTDTC','EGTEST','EGDTC'])

## reset index
aesi_t2_eg_b4 = aesi_t2_eg_b3.reset_index()

aesi_t2_eg_b5 = aesi_t2_eg_b4.copy() ## copy to new data frame

## set up initial variables
_usubjid = aesi_t2_eg_b5.loc[0,'USUBJID']
_aeterm = aesi_t2_eg_b5.loc[0,'AETERM']
_aedecod = aesi_t2_eg_b5.loc[0,'AEDECOD']
_aestdtc = aesi_t2_eg_b5.loc[0,'AESTDTC']
_egtest = aesi_t2_eg_b5.loc[0,'EGTEST']
last_index = aesi_t2_eg_b5.index[-1]

## find the last observation by usubjid, aeterm, aestdtc, egtest
for i, _df1 in aesi_t2_eg_b5.iterrows():
    
    ## assign last observation flag
    if i > 0:
        if _df1['USUBJID'] != _usubjid:
            _usubjid = _df1['USUBJID']
            aesi_t2_eg_b5.loc[i-1,'LAST_OBS'] = 1
        if _df1['AETERM'] != _aeterm:
            _aeterm = _df1['AETERM']
            aesi_t2_eg_b5.loc[i-1,'LAST_OBS'] = 1
        if _df1['AEDECOD'] != _aedecod:
            _aedecod = _df1['AEDECOD']
            aesi_t2_eg_b5.loc[i-1,'LAST_OBS'] = 1
        if _df1['AESTDTC'] != _aestdtc:
            _aestdtc = _df1['AESTDTC']
            aesi_t2_eg_b5.loc[i-1,'LAST_OBS'] = 1
        if _df1['EGTEST'] != _egtest:
            _egtest = _df1['EGTEST'] 
            aesi_t2_eg_b5.loc[i-1,'LAST_OBS'] = 1
            
    if i == last_index:
        aesi_t2_eg_b5.loc[i,'LAST_OBS'] = 1

## find non-last observation
for i, _df1 in aesi_t2_eg_b5.iterrows():
    if i > 0:
        if _df1['LAST_OBS'] == 1 :
            if ( _df1['USUBJID'] == aesi_t2_eg_b5.loc[i-1,'USUBJID'] ) and ( _df1['AETERM'] == aesi_t2_eg_b5.loc[i-1,'AETERM'] ) and ( _df1['AESTDTC'] == aesi_t2_eg_b5.loc[i-1,'AESTDTC'] ) and ( _df1['EGTEST'] != aesi_t2_eg_b5.loc[i-1,'EGTEST'] ) and ( _df1['EGDTC'] != aesi_t2_eg_b5.loc[i-1,'EGDTC'] ) : 
                aesi_t2_eg_b5.loc[i-1,'LAST_OBS'] = 0

aesi_t2_eg_b5[['USUBJID','AETERM','AESTDTC','EGTEST','EGDTC','LAST_OBS']]

,USUBJID,AETERM,AESTDTC,EGTEST,EGDTC,LAST_OBS
0,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,2018-01-03,MITRAL VALVE REGURGITATION,2018-01-03,1.0
1,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,2017-10-12,MITRAL VALVE REGURGITATION,2017-10-12,1.0
2,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,2017-10-24,MITRAL VALVE REGURGITATION,2017-10-24,1.0
3,ZX008-1501-0103-17,TRACE MITRAL REGURGITATION,2018-02-07,MITRAL VALVE REGURGITATION,2018-02-07,1.0
4,ZX008-1501-0104-03,MITRAL VALVE REGURGITATION TRACE,2018-01-02,MITRAL VALVE REGURGITATION,2017-12-11,1.0
5,ZX008-1501-0104-07,MITRAL VALVE REGURGITATION TRACE,2017-11-13,MITRAL VALVE REGURGITATION,2017-04-11,NaN
6,ZX008-1501-0104-07,MITRAL VALVE REGURGITATION TRACE,2017-11-13,MITRAL VALVE REGURGITATION,2017-04-11,NaN
7,ZX008-1501-0104-07,MITRAL VALVE REGURGITATION TRACE,2017-11-13,MITRAL VALVE REGURGITATION,2017-10-30,1.0
8,ZX008-1501-0104-09,MITRAL VALVE REGURGITATION TRACE,2017-08-31,MITRAL VALVE REGURGITATION,2017-08-11,1.0
9,ZX008-1501-0107-02,TRACE MITRAL VALVE REGURGITATION,2018-02-22,MITRAL VALVE REGURGITATION,2018-02-14,1.0


In [204]:
## Select the last observation  
aesi_t2_eg_b6 = aesi_t2_eg_b5[aesi_t2_eg_b5['LAST_OBS'] == 1]

aesi_t2_eg_b7 = aesi_t2_eg_b6.reset_index() ## convert index to columns
print(len(aesi_t2_eg_b7))
print(aesi_t2_eg_b7.columns.tolist())
aesi_t2_eg_b = aesi_t2_eg_b7[['STUDYID', 'USUBJID', 'SUBJID', 'SEX', 'AGE', 'AETERM', 'AEDECOD', 'AESTDTC', 'AEACN', 'AESEV', 
                'AESER', 'AEOUT', 'WEIGHTB', 'EGTEST', 'EGSTRESC', 'EGDTC', 'VISIT', 'ECHO']]
aesi_t2_eg_b

37
['level_0', 'index', 'STUDYID', 'USUBJID', 'SUBJID', 'SEX', 'AGE', 'AETERM', 'AEDECOD', 'AESTDTC', 'ASTDT', 'ASDY', 'AEACN', 'AESEV', 'AESER', 'AEOUT', 'TRTEMFL', 'WEIGHTB', 'EGTEST', 'EGSTRESC', 'EGDTC', 'VISIT', 'ECHO', 'LAST_OBS']


,STUDYID,USUBJID,SUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,AEACN,AESEV,AESER,AEOUT,WEIGHTB,EGTEST,EGSTRESC,EGDTC,VISIT,ECHO
0,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,DOSE NOT CHANGED,MILD,N,RECOVERED/RESOLVED,45.90,MITRAL VALVE REGURGITATION,TRACE,2018-01-03,OLE Visit 9 - Month 15,T
1,ZX008-1503,ZX008-1501-0101-09,0101-09,F,6.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,19.60,MITRAL VALVE REGURGITATION,TRACE,2017-10-12,OLE Visit 7 - Month 9,T
2,ZX008-1503,ZX008-1501-0101-14,0101-14,M,12.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-24,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,35.70,MITRAL VALVE REGURGITATION,TRACE,2017-10-24,OLE Visit 12 - Month 24,T
3,ZX008-1503,ZX008-1501-0103-17,0103-17,M,10.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-02-07,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,36.20,MITRAL VALVE REGURGITATION,TRACE,2018-02-07,OLE Visit 6 - Month 6,T
4,ZX008-1503,ZX008-1501-0104-03,0104-03,F,2.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2018-01-02,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,11.52,MITRAL VALVE REGURGITATION,TRACE,2017-12-11,OLE Visit 8 - Month 12,T
5,ZX008-1503,ZX008-1501-0104-07,0104-07,F,6.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-11-13,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,20.00,MITRAL VALVE REGURGITATION,TRACE,2017-10-30,OLE Visit 6 - Month 6,T
6,ZX008-1503,ZX008-1501-0104-09,0104-09,F,7.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-08-31,DOSE NOT CHANGED,MILD,N,RECOVERED/RESOLVED,27.99,MITRAL VALVE REGURGITATION,TRACE,2017-08-11,OLE Visit 3 - Month 1,T
7,ZX008-1503,ZX008-1501-0107-02,0107-02,M,8.0,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2018-02-22,DOSE NOT CHANGED,MILD,N,RECOVERING/RESOLVING,49.53,MITRAL VALVE REGURGITATION,TRACE,2018-02-14,OLE Visit 10 - Month 18,T
8,ZX008-1503,ZX008-1501-0107-04,0107-04,F,4.0,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-11-30,DOSE NOT CHANGED,MILD,N,RECOVERED/RESOLVED,15.29,MITRAL VALVE REGURGITATION,TRACE,2017-11-30,OLE Visit 8 - Month 12,T
9,ZX008-1503,ZX008-1501-0108-05,0108-05,M,11.0,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-12-12,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,40.91,MITRAL VALVE REGURGITATION,TRACE,2017-12-12,OLE Visit 3 - Month 1,T


In [205]:
## Create new variable - SUBSEQUENT ECHO reading
## Select EG after AESTDTC
aesi_t2_eg_a1 = aesi_t2_eg1[aesi_t2_eg1.AESTDTC < aesi_t2_eg1.EGDTC]
print("Length of aesi_t2_eg_a1 : ", len(aesi_t2_eg_a1))

## sort data
aesi_t2_eg_a2 = aesi_t2_eg_a1.sort_values(by=['USUBJID','AETERM','AEDECOD','AESTDTC','EGTEST','EGDTC'])

## reset index
aesi_t2_eg_a3 = aesi_t2_eg_a2.reset_index()

aesi_t2_eg_a4 = aesi_t2_eg_a3.copy() ## copy to new data frame

## set up initial variables
_echo2 = ''
_egtest = aesi_t2_eg_a4.loc[0,'EGTEST']
_usubjid = aesi_t2_eg_a4.loc[0,'USUBJID']
_aestdtc = aesi_t2_eg_a4.loc[0,'AESTDTC']
_aedecod = aesi_t2_eg_a4.loc[0,'AEDECOD']
_aeterm = aesi_t2_eg_a4.loc[0,'AETERM']
last_index = aesi_t2_eg_a4.index[-1]

## use for-loop to derive subsequent ECHO reading
for i, _df1 in aesi_t2_eg_a4.iterrows():
    ## create sub ECHO
    if i == 0:
        _echo2 = _df1['ECHO']
    else:
        _echo2 = _echo2 + '-' + _df1['ECHO']
    
    ## create sub echo and last observation for each AE & EG
    if i > 0:
        if _df1['USUBJID'] != _usubjid:
            _usubjid = _df1['USUBJID']
            _echo2 = _df1['ECHO']
            aesi_t2_eg_a4.loc[i-1,'LAST_OBS'] = 1
        if _df1['AETERM'] != _aeterm:
            _aeterm = _df1['AETERM']
            _echo2 = _df1['ECHO']
            aesi_t2_eg_a4.loc[i-1,'LAST_OBS'] = 1
        if _df1['AEDECOD'] != _aedecod:
            _aedecod = _df1['AEDECOD']
            _echo2 = _df1['ECHO']
            aesi_t2_eg_a4.loc[i-1,'LAST_OBS'] = 1
        if _df1['AESTDTC'] != _aestdtc:
            _aestdtc = _df1['AESTDTC']
            _echo2 = _df1['ECHO']
            aesi_t2_eg_a4.loc[i-1,'LAST_OBS'] = 1
        if _df1['EGTEST'] != _egtest:
            _egtest = _df1['EGTEST'] 
            _echo2 = _df1['ECHO']
            aesi_t2_eg_a4.loc[i-1,'LAST_OBS'] = 1
    if i == last_index:
        aesi_t2_eg_a4.loc[i,'LAST_OBS'] = 1
        
    ## Assign sub ECHO to data
    aesi_t2_eg_a4.loc[i,'ECHO_SUB'] = _echo2

## print variables
print(len(aesi_t2_eg_a4))
#print(aesi_t2_eg_a4[['USUBJID','AETERM','EGTEST','AESTDTC','EGDTC','ECHO','ECHO_SUB','LAST_OBS']])

Length of aesi_t2_eg_a1 :  86
86


In [206]:
## Pick up the last observation 
aesi_t2_eg_a5 = aesi_t2_eg_a4[aesi_t2_eg_a4.LAST_OBS ==1]
aesi_t2_eg_a = aesi_t2_eg_a5[['USUBJID','AETERM','EGTEST','ECHO_SUB']]
print(len(aesi_t2_eg_a))
#aesi_t2_eg_a

50


In [207]:
## merge previous dose with SUBSEQUENT ECHO reading 
aesi_t2_eg4 = pd.merge(aesi_t2_eg_b, aesi_t2_eg_a, how='left', on=['USUBJID','AETERM','EGTEST'])
aesi_t2_eg4['EGCODE']= aesi_t2_eg4.EGTEST.str[:6]
print(len(aesi_t2_eg4))
aesi_t2_eg4

37


,STUDYID,USUBJID,SUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,AEACN,AESEV,AESER,AEOUT,WEIGHTB,EGTEST,EGSTRESC,EGDTC,VISIT,ECHO,ECHO_SUB,EGCODE
0,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,DOSE NOT CHANGED,MILD,N,RECOVERED/RESOLVED,45.90,MITRAL VALVE REGURGITATION,TRACE,2018-01-03,OLE Visit 9 - Month 15,T,A,MITRAL
1,ZX008-1503,ZX008-1501-0101-09,0101-09,F,6.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,19.60,MITRAL VALVE REGURGITATION,TRACE,2017-10-12,OLE Visit 7 - Month 9,T,T-A,MITRAL
2,ZX008-1503,ZX008-1501-0101-14,0101-14,M,12.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-24,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,35.70,MITRAL VALVE REGURGITATION,TRACE,2017-10-24,OLE Visit 12 - Month 24,T,T,MITRAL
3,ZX008-1503,ZX008-1501-0103-17,0103-17,M,10.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-02-07,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,36.20,MITRAL VALVE REGURGITATION,TRACE,2018-02-07,OLE Visit 6 - Month 6,T,NaN,MITRAL
4,ZX008-1503,ZX008-1501-0104-03,0104-03,F,2.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2018-01-02,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,11.52,MITRAL VALVE REGURGITATION,TRACE,2017-12-11,OLE Visit 8 - Month 12,T,A,MITRAL
5,ZX008-1503,ZX008-1501-0104-07,0104-07,F,6.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-11-13,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,20.00,MITRAL VALVE REGURGITATION,TRACE,2017-10-30,OLE Visit 6 - Month 6,T,A,MITRAL
6,ZX008-1503,ZX008-1501-0104-09,0104-09,F,7.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-08-31,DOSE NOT CHANGED,MILD,N,RECOVERED/RESOLVED,27.99,MITRAL VALVE REGURGITATION,TRACE,2017-08-11,OLE Visit 3 - Month 1,T,A-A,MITRAL
7,ZX008-1503,ZX008-1501-0107-02,0107-02,M,8.0,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2018-02-22,DOSE NOT CHANGED,MILD,N,RECOVERING/RESOLVING,49.53,MITRAL VALVE REGURGITATION,TRACE,2018-02-14,OLE Visit 10 - Month 18,T,NaN,MITRAL
8,ZX008-1503,ZX008-1501-0107-04,0107-04,F,4.0,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-11-30,DOSE NOT CHANGED,MILD,N,RECOVERED/RESOLVED,15.29,MITRAL VALVE REGURGITATION,TRACE,2017-11-30,OLE Visit 8 - Month 12,T,A,MITRAL
9,ZX008-1503,ZX008-1501-0108-05,0108-05,M,11.0,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-12-12,DOSE NOT CHANGED,MILD,N,NOT RECOVERED/NOT RESOLVED,40.91,MITRAL VALVE REGURGITATION,TRACE,2017-12-12,OLE Visit 3 - Month 1,T,T,MITRAL


In [208]:
### Replace NaN to blank
aesi_t2_eg4['ECHO_SUB'] = aesi_t2_eg4['ECHO_SUB'].fillna(" ")
print(aesi_t2_eg4['ECHO_SUB'].head())


0      A
1    T-A
2      T
3       
4      A
Name: ECHO_SUB, dtype: object


In [209]:
### Pick up the last observation
## sort data
aesi_t2_eg5 = aesi_t2_eg4.sort_values(by=['USUBJID','AETERM','AEDECOD','AESTDTC','EGDTC','EGTEST'])

## reset index
aesi_t2_eg6 = aesi_t2_eg5.reset_index()

aesi_t2_eg7 = aesi_t2_eg6.copy() ## copy to new data frame

## set up initial variables
_usubjid = aesi_t2_eg7.loc[0,'USUBJID']
_aestdtc = aesi_t2_eg7.loc[0,'AESTDTC']
_aedecod = aesi_t2_eg7.loc[0,'AEDECOD']
_aeterm = aesi_t2_eg7.loc[0,'AETERM']
_egdtc = aesi_t2_eg7.loc[0,'EGDTC']
last_index = aesi_t2_eg7.index[-1]

## use for-loop to identify the last observation
for i, _df1 in aesi_t2_eg7.iterrows():
       
    if i > 0:
        ## last observation 
        if _df1['USUBJID'] != _usubjid:
            _usubjid = _df1['USUBJID']           
            aesi_t2_eg7.loc[i-1,'LAST_OBS'] = 1

        if _df1['AETERM'] != _aeterm:
            _aeterm = _df1['AETERM']          
            aesi_t2_eg7.loc[i-1,'LAST_OBS'] = 1

        if _df1['AEDECOD'] != _aedecod:
            _aedecod = _df1['AEDECOD']           
            aesi_t2_eg7.loc[i-1,'LAST_OBS'] = 1

        if _df1['AESTDTC'] != _aestdtc:
            _aestdtc = _df1['AESTDTC']           
            aesi_t2_eg7.loc[i-1,'LAST_OBS'] = 1

        if _df1['EGDTC'] != _egdtc:
            _egdtc = _df1['EGDTC']            
            aesi_t2_eg7.loc[i-1,'LAST_OBS'] = 1
                    
    ## end of observation
    if i == last_index:
        aesi_t2_eg7.loc[i,'LAST_OBS'] = 1

## create new ECG Test and Subsequent ECHO at the last observation        
for i, _df1 in aesi_t2_eg7.iterrows():
       
    ## At last observation 
    if _df1['LAST_OBS'] == 1:
        if i > 0:
            ## compare with the previous date - combine 
            if (_df1['EGDTC'] ==  aesi_t2_eg7.loc[i-1,'EGDTC']):
                aesi_t2_eg7.loc[i,'ECGTEST'] = aesi_t2_eg7.loc[i-1,'EGTEST'] + '/' + aesi_t2_eg7.loc[i,'EGTEST'] 
                aesi_t2_eg7.loc[i,'ECHO_SUB2'] = aesi_t2_eg7.loc[i-1,'ECHO_SUB'] + '/' + aesi_t2_eg7.loc[i,'ECHO_SUB']
            else:
                aesi_t2_eg7.loc[i,'ECGTEST'] = aesi_t2_eg7.loc[i,'EGTEST'] 
                aesi_t2_eg7.loc[i,'ECHO_SUB2'] = aesi_t2_eg7.loc[i,'ECHO_SUB']
        elif i == 0:
            aesi_t2_eg7.loc[i,'ECGTEST'] = aesi_t2_eg7.loc[i,'EGTEST'] 
            aesi_t2_eg7.loc[i,'ECHO_SUB2'] = aesi_t2_eg7.loc[i,'ECHO_SUB']
                    
print(len(aesi_t2_eg7))
aesi_t2_eg7

37


,index,STUDYID,USUBJID,SUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,AEACN,...,EGTEST,EGSTRESC,EGDTC,VISIT,ECHO,ECHO_SUB,EGCODE,LAST_OBS,ECGTEST,ECHO_SUB2
0,0,ZX008-1503,ZX008-1501-0101-02,0101-02,M,16.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,DOSE NOT CHANGED,...,MITRAL VALVE REGURGITATION,TRACE,2018-01-03,OLE Visit 9 - Month 15,T,A,MITRAL,1.0,MITRAL VALVE REGURGITATION,A
1,1,ZX008-1503,ZX008-1501-0101-09,0101-09,F,6.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,DOSE NOT CHANGED,...,MITRAL VALVE REGURGITATION,TRACE,2017-10-12,OLE Visit 7 - Month 9,T,T-A,MITRAL,1.0,MITRAL VALVE REGURGITATION,T-A
2,2,ZX008-1503,ZX008-1501-0101-14,0101-14,M,12.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-24,DOSE NOT CHANGED,...,MITRAL VALVE REGURGITATION,TRACE,2017-10-24,OLE Visit 12 - Month 24,T,T,MITRAL,1.0,MITRAL VALVE REGURGITATION,T
3,3,ZX008-1503,ZX008-1501-0103-17,0103-17,M,10.0,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-02-07,DOSE NOT CHANGED,...,MITRAL VALVE REGURGITATION,TRACE,2018-02-07,OLE Visit 6 - Month 6,T,,MITRAL,1.0,MITRAL VALVE REGURGITATION,
4,4,ZX008-1503,ZX008-1501-0104-03,0104-03,F,2.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2018-01-02,DOSE NOT CHANGED,...,MITRAL VALVE REGURGITATION,TRACE,2017-12-11,OLE Visit 8 - Month 12,T,A,MITRAL,1.0,MITRAL VALVE REGURGITATION,A
5,5,ZX008-1503,ZX008-1501-0104-07,0104-07,F,6.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-11-13,DOSE NOT CHANGED,...,MITRAL VALVE REGURGITATION,TRACE,2017-10-30,OLE Visit 6 - Month 6,T,A,MITRAL,1.0,MITRAL VALVE REGURGITATION,A
6,6,ZX008-1503,ZX008-1501-0104-09,0104-09,F,7.0,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-08-31,DOSE NOT CHANGED,...,MITRAL VALVE REGURGITATION,TRACE,2017-08-11,OLE Visit 3 - Month 1,T,A-A,MITRAL,1.0,MITRAL VALVE REGURGITATION,A-A
7,7,ZX008-1503,ZX008-1501-0107-02,0107-02,M,8.0,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2018-02-22,DOSE NOT CHANGED,...,MITRAL VALVE REGURGITATION,TRACE,2018-02-14,OLE Visit 10 - Month 18,T,,MITRAL,1.0,MITRAL VALVE REGURGITATION,
8,8,ZX008-1503,ZX008-1501-0107-04,0107-04,F,4.0,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-11-30,DOSE NOT CHANGED,...,MITRAL VALVE REGURGITATION,TRACE,2017-11-30,OLE Visit 8 - Month 12,T,A,MITRAL,1.0,MITRAL VALVE REGURGITATION,A
9,9,ZX008-1503,ZX008-1501-0108-05,0108-05,M,11.0,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-12-12,DOSE NOT CHANGED,...,MITRAL VALVE REGURGITATION,TRACE,2017-12-12,OLE Visit 3 - Month 1,T,T,MITRAL,1.0,MITRAL VALVE REGURGITATION,T


In [210]:
aesi_t2_eg8 = aesi_t2_eg7[aesi_t2_eg7.LAST_OBS == 1]
print(len(aesi_t2_eg8))

35


In [211]:
aesi_t2_eg = aesi_t2_eg8[['STUDYID','USUBJID', 'AETERM','AEDECOD','AESTDTC','ECGTEST','EGDTC','SEX','AGE','WEIGHTB','AEACN','AESER','ECHO_SUB2']]
print(len(aesi_t2_eg))
aesi_t2_eg

35


,STUDYID,USUBJID,AETERM,AEDECOD,AESTDTC,ECGTEST,EGDTC,SEX,AGE,WEIGHTB,AEACN,AESER,ECHO_SUB2
0,ZX008-1503,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,MITRAL VALVE REGURGITATION,2018-01-03,M,16.0,45.90,DOSE NOT CHANGED,N,A
1,ZX008-1503,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,MITRAL VALVE REGURGITATION,2017-10-12,F,6.0,19.60,DOSE NOT CHANGED,N,T-A
2,ZX008-1503,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-24,MITRAL VALVE REGURGITATION,2017-10-24,M,12.0,35.70,DOSE NOT CHANGED,N,T
3,ZX008-1503,ZX008-1501-0103-17,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-02-07,MITRAL VALVE REGURGITATION,2018-02-07,M,10.0,36.20,DOSE NOT CHANGED,N,
4,ZX008-1503,ZX008-1501-0104-03,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2018-01-02,MITRAL VALVE REGURGITATION,2017-12-11,F,2.0,11.52,DOSE NOT CHANGED,N,A
5,ZX008-1503,ZX008-1501-0104-07,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-11-13,MITRAL VALVE REGURGITATION,2017-10-30,F,6.0,20.00,DOSE NOT CHANGED,N,A
6,ZX008-1503,ZX008-1501-0104-09,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-08-31,MITRAL VALVE REGURGITATION,2017-08-11,F,7.0,27.99,DOSE NOT CHANGED,N,A-A
7,ZX008-1503,ZX008-1501-0107-02,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2018-02-22,MITRAL VALVE REGURGITATION,2018-02-14,M,8.0,49.53,DOSE NOT CHANGED,N,
8,ZX008-1503,ZX008-1501-0107-04,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-11-30,MITRAL VALVE REGURGITATION,2017-11-30,F,4.0,15.29,DOSE NOT CHANGED,N,A
9,ZX008-1503,ZX008-1501-0108-05,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-12-12,MITRAL VALVE REGURGITATION,2017-12-12,M,11.0,40.91,DOSE NOT CHANGED,N,T


### Merge with Exposure - EC

In [273]:
## merge with Exposure EX
aesi_t2_eg_ex1 = pd.merge(aesi_t2_eg, _ex, how='left', on='USUBJID')
print(len(aesi_t2_eg_ex1))
aesi_t2_eg_ex1[['USUBJID','AETERM','AESTDTC','EGDTC','ECGTEST','EXSTDTC','EXENDTC']]

98


,USUBJID,AETERM,AESTDTC,EGDTC,ECGTEST,EXSTDTC,EXENDTC
0,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,2018-01-03,2018-01-03,MITRAL VALVE REGURGITATION,2016-10-05,2016-11-01
1,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,2018-01-03,2018-01-03,MITRAL VALVE REGURGITATION,2016-11-02,2017-01-03
2,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,2018-01-03,2018-01-03,MITRAL VALVE REGURGITATION,2017-01-04,2018-06-14
3,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,2017-10-12,2017-10-12,MITRAL VALVE REGURGITATION,2017-01-18,2017-02-14
4,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,2017-10-12,2017-10-12,MITRAL VALVE REGURGITATION,2017-02-15,2017-03-14
5,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,2017-10-12,2017-10-12,MITRAL VALVE REGURGITATION,2017-03-15,2017-04-05
6,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,2017-10-12,2017-10-12,MITRAL VALVE REGURGITATION,2017-04-06,2018-06-17
7,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,2017-10-24,2017-10-24,MITRAL VALVE REGURGITATION,2017-03-14,2017-04-06
8,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,2017-10-24,2017-10-24,MITRAL VALVE REGURGITATION,2017-04-07,2017-06-06
9,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,2017-10-24,2017-10-24,MITRAL VALVE REGURGITATION,2017-06-07,2017-10-21


In [274]:
## Keep only the records that EXSTDTC < EGDTC
aesi_t2_eg_ex2 = aesi_t2_eg_ex1[aesi_t2_eg_ex1.EXSTDTC < aesi_t2_eg_ex1.EGDTC]
print(len(aesi_t2_eg_ex2))
print(aesi_t2_eg_ex2.columns.tolist())
#print(aesi_t2_eg_ex2[['EGDTC','EXSTDTC','EXENDTC']][:20])

80
['STUDYID', 'USUBJID', 'AETERM', 'AEDECOD', 'AESTDTC', 'ECGTEST', 'EGDTC', 'SEX', 'AGE', 'WEIGHTB', 'AEACN', 'AESER', 'ECHO_SUB2', 'EXDOSTXT', 'EXSTDTC', 'EXENDTC']


In [275]:
## Create days of exposure from EG and days of exposure
## import date modules
import datetime as dt

## sort data;
aesi_t2_eg_ex2_sorted = aesi_t2_eg_ex2.sort_values(by=['USUBJID','AEDECOD','AESTDTC','ECGTEST','EGDTC','EXSTDTC'])

## copy to new data frame
aesi_t2_eg_ex3 = aesi_t2_eg_ex2_sorted.copy() 

## use for-loop to derive days of exposure
for i, _df2 in aesi_t2_eg_ex3.iterrows():
    # Derive Days of differnet doses and Days of exposures from EG date
    exstdtc = dt.datetime.strptime(_df2.EXSTDTC, '%Y-%m-%d')
    exendtc = dt.datetime.strptime(_df2.EXENDTC, '%Y-%m-%d')
    egdtc = dt.datetime.strptime(_df2.EGDTC, '%Y-%m-%d')
    
    if _df2.EGDTC > _df2.EXENDTC:
        aesi_t2_eg_ex3.loc[i, 'DAYS'] = (exendtc - exstdtc).days + 1
        aesi_t2_eg_ex3.loc[i, 'EG_EXSTDY'] = (exendtc - exstdtc).days + 1 ## Days of drugs from EGDTC
    else:
        aesi_t2_eg_ex3.loc[i, 'DAYS'] = (egdtc - exstdtc).days
        aesi_t2_eg_ex3.loc[i, 'EG_EXSTDY'] = (egdtc - exstdtc).days  ## Days of drugs from EGDTC
            
print(len(aesi_t2_eg_ex3))
#print(aesi_t2_eg_ex3[['EGDTC','EXSTDTC','EXENDTC','DAYS']][:20])
#aesi_t2_eg_ex3[['USUBJID','EGDTC','EXSTDTC','EXENDTC','DAYS']]

80


In [276]:
## Find the last observation by USUBJID, AETERM, AEDECOD, AESTDTC and EGDTC
## sort data
aesi_t2_eg_ex3_sorted = aesi_t2_eg_ex3.sort_values(by=['USUBJID','AETERM','AEDECOD','AESTDTC','EGDTC'])

## reset index for loop iteration
aesi_t2_eg_ex3_reset = aesi_t2_eg_ex3_sorted.reset_index()
## copy to new data frame
aesi_t2_eg_ex4 = aesi_t2_eg_ex3_reset.copy() 

## Set up initial variables
_usubjid = aesi_t2_eg_ex4.loc[0,'USUBJID']
_aeterm = aesi_t2_eg_ex4.loc[0,'AETERM']
_aestdtc = aesi_t2_eg_ex4.loc[0,'AESTDTC']
_egdtc = aesi_t2_eg_ex4.loc[0,'EGDTC']

last_index = aesi_t2_eg_ex4.index[-1] ## last index number 

## use for-loop to derive days of exposure
for i, _df2 in aesi_t2_eg_ex4.iterrows():
    
    #Last observation of each group. 
    if i > 0:
        if _df2['USUBJID'] != _usubjid:
            aesi_t2_eg_ex4.loc[i-1, 'LAST_OBS'] = 1
            _usubjid = _df2['USUBJID']
            aesi_t2_eg_ex4.loc[i, 'TEST'] = 1
        if _df2['AESTDTC'] != _aestdtc:
            aesi_t2_eg_ex4.loc[i-1, 'LAST_OBS'] = 1
            _aestdtc = _df2['AESTDTC']
            aesi_t2_eg_ex4.loc[i, 'TEST'] = 2
        if _df2['AETERM'] != _aeterm:
            aesi_t2_eg_ex4.loc[i-1, 'LAST_OBS'] = 1
            _aeterm = _df2['AETERM']
            aesi_t2_eg_ex4.loc[i, 'TEST'] = 3
        if _df2['EGDTC'] != _egdtc:
            aesi_t2_eg_ex4.loc[i-1, 'LAST_OBS'] = 1
            _egdtc = _df2['EGDTC']
            aesi_t2_eg_ex4.loc[i, 'TEST'] = 4

    if i == (last_index):  # for last record  
        aesi_t2_eg_ex4.loc[i, 'LAST_OBS'] = 1

aesi_t2_eg_ex = aesi_t2_eg_ex4.drop(columns=['index'])
print(len(aesi_t2_eg_ex))
#aesi_t2_eg_ex
aesi_t2_eg_ex[['STUDYID','USUBJID','AETERM','AESTDTC','ECGTEST','EGDTC','LAST_OBS']]

80


,STUDYID,USUBJID,AETERM,AESTDTC,ECGTEST,EGDTC,LAST_OBS
0,ZX008-1503,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,2018-01-03,MITRAL VALVE REGURGITATION,2018-01-03,NaN
1,ZX008-1503,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,2018-01-03,MITRAL VALVE REGURGITATION,2018-01-03,NaN
2,ZX008-1503,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,2018-01-03,MITRAL VALVE REGURGITATION,2018-01-03,1.0
3,ZX008-1503,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,2017-10-12,MITRAL VALVE REGURGITATION,2017-10-12,NaN
4,ZX008-1503,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,2017-10-12,MITRAL VALVE REGURGITATION,2017-10-12,NaN
5,ZX008-1503,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,2017-10-12,MITRAL VALVE REGURGITATION,2017-10-12,NaN
6,ZX008-1503,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,2017-10-12,MITRAL VALVE REGURGITATION,2017-10-12,1.0
7,ZX008-1503,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,2017-10-24,MITRAL VALVE REGURGITATION,2017-10-24,NaN
8,ZX008-1503,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,2017-10-24,MITRAL VALVE REGURGITATION,2017-10-24,NaN
9,ZX008-1503,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,2017-10-24,MITRAL VALVE REGURGITATION,2017-10-24,1.0


### Create Last dose 

In [277]:
## Select Last dose per each SUBJECT, AESTDTC, EGTEST
aesi_t2_eg_ex_last = aesi_t2_eg_ex[aesi_t2_eg_ex.LAST_OBS == 1]
print(len(aesi_t2_eg_ex_last))
aesi_t2_eg_ex_last

35


,STUDYID,USUBJID,AETERM,AEDECOD,AESTDTC,ECGTEST,EGDTC,SEX,AGE,WEIGHTB,AEACN,AESER,ECHO_SUB2,EXDOSTXT,EXSTDTC,EXENDTC,DAYS,EG_EXSTDY,LAST_OBS,TEST
2,ZX008-1503,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,MITRAL VALVE REGURGITATION,2018-01-03,M,16.0,45.90,DOSE NOT CHANGED,N,A,0.8,2017-01-04,2018-06-14,364.0,364.0,1.0,NaN
6,ZX008-1503,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,MITRAL VALVE REGURGITATION,2017-10-12,F,6.0,19.60,DOSE NOT CHANGED,N,T-A,0.8,2017-04-06,2018-06-17,189.0,189.0,1.0,NaN
9,ZX008-1503,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-24,MITRAL VALVE REGURGITATION,2017-10-24,M,12.0,35.70,DOSE NOT CHANGED,N,T,0.6,2017-06-07,2017-10-21,137.0,137.0,1.0,NaN
13,ZX008-1503,ZX008-1501-0103-17,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-02-07,MITRAL VALVE REGURGITATION,2018-02-07,M,10.0,36.20,DOSE NOT CHANGED,N,,0.8,2017-10-05,2018-06-17,125.0,125.0,1.0,NaN
17,ZX008-1503,ZX008-1501-0104-03,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2018-01-02,MITRAL VALVE REGURGITATION,2017-12-11,F,2.0,11.52,DOSE NOT CHANGED,N,A,0.8,2017-03-15,2018-06-16,271.0,271.0,1.0,NaN
19,ZX008-1503,ZX008-1501-0104-07,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-11-13,MITRAL VALVE REGURGITATION,2017-10-30,F,6.0,20.00,DOSE NOT CHANGED,N,A,0.4,2017-07-03,2018-01-21,119.0,119.0,1.0,NaN
20,ZX008-1503,ZX008-1501-0104-09,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-08-31,MITRAL VALVE REGURGITATION,2017-08-11,F,7.0,27.99,DOSE NOT CHANGED,N,A-A,0.2,2017-06-23,2019-01-26,49.0,49.0,1.0,4.0
21,ZX008-1503,ZX008-1501-0107-02,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2018-02-22,MITRAL VALVE REGURGITATION,2018-02-14,M,8.0,49.53,DOSE NOT CHANGED,N,,0.2,2016-09-14,2018-06-17,518.0,518.0,1.0,4.0
25,ZX008-1503,ZX008-1501-0107-04,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-11-30,MITRAL VALVE REGURGITATION,2017-11-30,F,4.0,15.29,DOSE NOT CHANGED,N,A,0.8,2017-09-06,2018-06-06,85.0,85.0,1.0,NaN
26,ZX008-1503,ZX008-1501-0108-05,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-12-12,MITRAL VALVE REGURGITATION,2017-12-12,M,11.0,40.91,DOSE NOT CHANGED,N,T,0.2,2017-11-09,2017-12-11,33.0,33.0,1.0,4.0


### Create Previous Doses

In [278]:
## Select non-last dose 
aesi_t2_eg_ex_last_non = aesi_t2_eg_ex[aesi_t2_eg_ex.LAST_OBS != 1]
print(len(aesi_t2_eg_ex_last_non))
print(aesi_t2_eg_ex_last_non.columns.tolist())

45
['STUDYID', 'USUBJID', 'AETERM', 'AEDECOD', 'AESTDTC', 'ECGTEST', 'EGDTC', 'SEX', 'AGE', 'WEIGHTB', 'AEACN', 'AESER', 'ECHO_SUB2', 'EXDOSTXT', 'EXSTDTC', 'EXENDTC', 'DAYS', 'EG_EXSTDY', 'LAST_OBS', 'TEST']


In [279]:
## Derive the sum of days based on doses grouping by USUBJID, AETERM,AEDECOD, AESTDTC, EGTEST on DAYS 
aesi_t2_eg_ex_prev1= aesi_t2_eg_ex_last_non['DAYS'].groupby([aesi_t2_eg_ex_last_non['STUDYID'], 
    aesi_t2_eg_ex_last_non['USUBJID'],aesi_t2_eg_ex_last_non['AETERM'], aesi_t2_eg_ex_last_non['AEDECOD'], 
    aesi_t2_eg_ex_last_non['AESTDTC'], aesi_t2_eg_ex_last_non['ECGTEST'], aesi_t2_eg_ex_last_non['EXDOSTXT']]).sum()
aesi_t2_eg_ex_prev2 = aesi_t2_eg_ex_prev1.reset_index() ## convert index to columns
print(len(aesi_t2_eg_ex_prev2))
print(aesi_t2_eg_ex_prev2[['USUBJID','AETERM','ECGTEST','EXDOSTXT','DAYS']])

45
                  USUBJID                                             AETERM  \
0      ZX008-1501-0101-02                         TRACE MITRAL REGURGITATION   
1      ZX008-1501-0101-02                         TRACE MITRAL REGURGITATION   
2      ZX008-1501-0101-09                         TRACE MITRAL REGURGITATION   
3      ZX008-1501-0101-09                         TRACE MITRAL REGURGITATION   
4      ZX008-1501-0101-09                         TRACE MITRAL REGURGITATION   
5      ZX008-1501-0101-14                         TRACE MITRAL REGURGITATION   
6      ZX008-1501-0101-14                         TRACE MITRAL REGURGITATION   
7      ZX008-1501-0103-17                         TRACE MITRAL REGURGITATION   
8      ZX008-1501-0103-17                         TRACE MITRAL REGURGITATION   
9      ZX008-1501-0103-17                         TRACE MITRAL REGURGITATION   
10     ZX008-1501-0104-03                   MITRAL VALVE REGURGITATION TRACE   
11     ZX008-1501-0104-03            

In [280]:
aesi_t2_eg_ex_prev2['EXDOSTXT'].value_counts()

0.2    23
0.4    16
0.6     6
Name: EXDOSTXT, dtype: int64

In [281]:
## Create new variable - DOSE for treatment
aesi_t2_eg_ex_prev3 = aesi_t2_eg_ex_prev2.copy()

for i, _df4 in aesi_t2_eg_ex_prev3.iterrows():
    if _df4['EXDOSTXT'] == 0.2 :
        aesi_t2_eg_ex_prev3.loc[i, 'DOSE'] = '0.2mg'
    elif _df4['EXDOSTXT'] == 0.4 :
        aesi_t2_eg_ex_prev3.loc[i, 'DOSE'] = '0.4mg'
    elif _df4['EXDOSTXT'] == 0.6 :
        aesi_t2_eg_ex_prev3.loc[i, 'DOSE'] = '0.6mg'

print(len(aesi_t2_eg_ex_prev3))
print(aesi_t2_eg_ex_prev3.columns.tolist())
aesi_t2_eg_ex_prev3

45
['STUDYID', 'USUBJID', 'AETERM', 'AEDECOD', 'AESTDTC', 'ECGTEST', 'EXDOSTXT', 'DAYS', 'DOSE']


,STUDYID,USUBJID,AETERM,AEDECOD,AESTDTC,ECGTEST,EXDOSTXT,DAYS,DOSE
0,ZX008-1503,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,MITRAL VALVE REGURGITATION,0.2,28.0,0.2mg
1,ZX008-1503,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,MITRAL VALVE REGURGITATION,0.6,63.0,0.6mg
2,ZX008-1503,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,MITRAL VALVE REGURGITATION,0.2,28.0,0.2mg
3,ZX008-1503,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,MITRAL VALVE REGURGITATION,0.4,28.0,0.4mg
4,ZX008-1503,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,MITRAL VALVE REGURGITATION,0.6,22.0,0.6mg
5,ZX008-1503,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-24,MITRAL VALVE REGURGITATION,0.2,24.0,0.2mg
6,ZX008-1503,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-24,MITRAL VALVE REGURGITATION,0.4,61.0,0.4mg
7,ZX008-1503,ZX008-1501-0103-17,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-02-07,MITRAL VALVE REGURGITATION,0.2,14.0,0.2mg
8,ZX008-1503,ZX008-1501-0103-17,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-02-07,MITRAL VALVE REGURGITATION,0.4,17.0,0.4mg
9,ZX008-1503,ZX008-1501-0103-17,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-02-07,MITRAL VALVE REGURGITATION,0.6,28.0,0.6mg


In [282]:
### Create Dose days variable for 02, 04, 08 mg and placebo
aesi_t2_eg_ex_prev4 = pd.pivot_table(aesi_t2_eg_ex_prev3, values='DAYS', columns = ['DOSE'], 
                                     index=['USUBJID','AETERM','AEDECOD','AESTDTC','ECGTEST'])
aesi_t2_eg_ex_prev = aesi_t2_eg_ex_prev4.reset_index() # reset index
print(len(aesi_t2_eg_ex_prev))
aesi_t2_eg_ex_prev

23


DOSE,USUBJID,AETERM,AEDECOD,AESTDTC,ECGTEST,0.2mg,0.4mg,0.6mg
0,ZX008-1501-0101-02,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-03,MITRAL VALVE REGURGITATION,28.0,NaN,63.0
1,ZX008-1501-0101-09,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-12,MITRAL VALVE REGURGITATION,28.0,28.0,22.0
2,ZX008-1501-0101-14,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2017-10-24,MITRAL VALVE REGURGITATION,24.0,61.0,NaN
3,ZX008-1501-0103-17,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-02-07,MITRAL VALVE REGURGITATION,14.0,17.0,28.0
4,ZX008-1501-0104-03,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2018-01-02,MITRAL VALVE REGURGITATION,30.0,34.0,28.0
5,ZX008-1501-0104-07,MITRAL VALVE REGURGITATION TRACE,Echocardiogram abnormal,2017-11-13,MITRAL VALVE REGURGITATION,182.0,NaN,NaN
6,ZX008-1501-0107-04,TRACE MITRAL VALVE REGURGITATION,Echocardiogram abnormal,2017-11-30,MITRAL VALVE REGURGITATION,89.0,92.0,90.0
7,ZX008-1501-0109-04,ECHOCARDIOGRAM ABNORMAL,Echocardiogram abnormal,2018-02-01,MITRAL VALVE REGURGITATION,28.0,26.0,NaN
8,ZX008-1501-0109-05,ECHOCARDIOGRAM ABNORMAL: INTERMITTENT TRACE MI...,Echocardiogram abnormal,2017-11-07,MITRAL VALVE REGURGITATION,28.0,26.0,NaN
9,ZX008-1501-0110-04,TRACE MITRAL REGURGITATION,Echocardiogram abnormal,2018-01-05,MITRAL VALVE REGURGITATION,32.0,NaN,NaN


### Combine last does and previous dose for final report

In [283]:
## since there is no record in aesi_t2_eg_ex_prev, we don't merge with it
aesi_t2_eg_ex_final = pd.merge(aesi_t2_eg_ex_last, aesi_t2_eg_ex_prev, how='left', 
                                on=['USUBJID','AETERM','AEDECOD','AESTDTC','ECGTEST']) 
#aesi_t2_eg_ex_final = aesi_t2_eg_ex_last
print(len(aesi_t2_eg_ex_final))
#aesi_t2_eg_ex_final

35


In [284]:
aesi_t2_final = aesi_t2_eg_ex_final
print(aesi_t2_final.columns.tolist())

['STUDYID', 'USUBJID', 'AETERM', 'AEDECOD', 'AESTDTC', 'ECGTEST', 'EGDTC', 'SEX', 'AGE', 'WEIGHTB', 'AEACN', 'AESER', 'ECHO_SUB2', 'EXDOSTXT', 'EXSTDTC', 'EXENDTC', 'DAYS', 'EG_EXSTDY', 'LAST_OBS', 'TEST', '0.2mg', '0.4mg', '0.6mg']


## Table 3

### Prepare SDTM LB(Lab) datasets

In [285]:
### Read SDTM LB datasets
with open('../sdtm/lb.xpt', 'rb') as _lb1: 
    _lb2 = xport.to_columns(_lb1)
    _lb3 = pd.DataFrame(_lb2)

In [286]:
_lb4 = _lb3[_lb3.LBTESTCD == 'PROLCTN']
_lb = _lb4[['USUBJID','LBTESTCD','LBSTRESC','LBSTRESU','LBSTNRLO','LBSTNRHI','LBDTC','VISIT']]
print(len(_lb))

1314


### Prepare SDTM XS(Seizures) datasets

In [287]:
### Read SDTM XS datasets
with open('../sdtm/xs.xpt', 'rb') as _xs1: 
    _xs2 = xport.to_columns(_xs1)
    _xs3 = pd.DataFrame(_xs2)

In [288]:
a = _xs3.XSTEST.value_counts()
b = a.sort_index()
print(type(a))
print(b)

<class 'pandas.core.series.Series'>
ABSENCE                                        367
ABSENCE DESCRIPTION                            367
ADDITIONAL 1 TOGETHER DELAYED                  806
ADDITIONAL 2 TOGETHER DELAYED                  348
ADDITIONAL 3 TOGETHER DELAYED                  193
ADDITIONAL 4 TOGETHER DELAYED                  139
ADDITIONAL RESCUE MEDICATION 2 GIVEN          4565
ADDITIONAL RESCUE MEDICATION 3 GIVEN           399
ADDITIONAL RESCUE MEDICATION 4 GIVEN           192
ADDITIONAL RESCUE MEDICATION GIVEN           12025
ANOTHER SEIZURE TO REPORT                    28527
APPROXIMATE TOTAL TIME OF CLUSTER            37806
ATONIC                                          93
ATONIC DESCRIPTION                              93
CLONIC                                         122
CLONIC DESCRIPTION                             122
CLUSTER OF SEIZURES NEXT QUESTION              262
END OF DAY REVIEW COMPLIANCE                   215
EPILEPTIC                                     

In [289]:
_xs4 = _xs3[(_xs3.XSTESTCD == 'TIMSEZ1L')]
_xs5 = _xs4[['USUBJID','XSTESTCD','XSTEST','XSORRES','XSDTC','XSSTDTC']]
_xs = _xs5.sort_values(by=['USUBJID','XSTESTCD','XSORRES','XSDTC','XSSTDTC'])
#_xs7 = pd.crosstab(_xs6, columns=['XSTESTCD'], values='XSORRES', index=['USUBJID','XSDTC','XSSTDTC'])
#_xs[['USUBJID','XSORRES']].value_counts()
#_xs['XSTESTCD'].groupby([ _xs['XSTEST'], _xs['XSORRES']]).count()
print(len(_xs))

136858


### AESI Data for Table 3 

In [290]:
adae_sl['AEDECOD'].value_counts()

Echocardiogram abnormal               34
Blood glucose decreased               14
Blood pressure diastolic increased     7
Blood pressure increased               6
Heart rate increased                   3
Hypoglycaemia                          2
Tachycardia                            2
Blood prolactin increased              2
Blood pressure systolic increased      1
Dysphagia                              1
Mitral valve incompetence              1
Cardiac murmur                         1
Name: AEDECOD, dtype: int64

In [291]:
## Subsetting AESI for 'Blood prolactin increased'
aesi_t3_1 = adae_sl[adae_sl.AEDECOD == 'Blood prolactin increased']
aesi_t3 = aesi_t3_1[['STUDYID','USUBJID','SEX','AGE','AETERM','AEDECOD','AESTDTC','AEACN','AESER','AESEV','WEIGHTB','AEOUT']]
print(len(aesi_t3))
print(aesi_t3)

2
       STUDYID             USUBJID SEX  AGE  \
8   ZX008-1503  ZX008-1501-0103-11   F  5.0   
32  ZX008-1503  ZX008-1501-0118-04   F  2.0   

                                AETERM                    AEDECOD     AESTDTC  \
8   V8 PROLACTIN IS 94.9 H (INCREASED)  Blood prolactin increased  2018-02-08   
32            ELEVATED PROLACTIN LEVEL  Blood prolactin increased  2018-01-25   

               AEACN AESER AESEV  WEIGHTB                       AEOUT  
8   DOSE NOT CHANGED     N  MILD    25.72          RECOVERED/RESOLVED  
32  DOSE NOT CHANGED     N  MILD    15.29  NOT RECOVERED/NOT RESOLVED  


### Merge AESI of table 3 with SDTM LB (Lab)

In [292]:
## Merge AESI of Table 3 with SDTM LB
aesi_t3_lb1 = pd.merge(aesi_t3, _lb, how='left', on='USUBJID')
print(len(aesi_t3_lb1))
aesi_t3_lb1

12


,STUDYID,USUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,AEACN,AESER,AESEV,WEIGHTB,AEOUT,LBTESTCD,LBSTRESC,LBSTRESU,LBSTNRLO,LBSTNRHI,LBDTC,VISIT
0,ZX008-1503,ZX008-1501-0103-11,F,5.0,V8 PROLACTIN IS 94.9 H (INCREASED),Blood prolactin increased,2018-02-08,DOSE NOT CHANGED,N,MILD,25.72,RECOVERED/RESOLVED,PROLCTN,1791.4,mIU/L,44.5,375.2,2017-01-25T08:59:00,Visit 12 (Clinic)
1,ZX008-1503,ZX008-1501-0103-11,F,5.0,V8 PROLACTIN IS 94.9 H (INCREASED),Blood prolactin increased,2018-02-08,DOSE NOT CHANGED,N,MILD,25.72,RECOVERED/RESOLVED,PROLCTN,61.5,mIU/L,44.5,375.2,2017-03-09T07:49:00,OLE Visit 3 - Month 1
2,ZX008-1503,ZX008-1501-0103-11,F,5.0,V8 PROLACTIN IS 94.9 H (INCREASED),Blood prolactin increased,2018-02-08,DOSE NOT CHANGED,N,MILD,25.72,RECOVERED/RESOLVED,PROLCTN,1863.5,mIU/L,44.5,375.2,2017-04-06T09:17:00,OLE Visit 4 - Month 2
3,ZX008-1503,ZX008-1501-0103-11,F,5.0,V8 PROLACTIN IS 94.9 H (INCREASED),Blood prolactin increased,2018-02-08,DOSE NOT CHANGED,N,MILD,25.72,RECOVERED/RESOLVED,PROLCTN,1835.9,mIU/L,44.5,375.2,2017-05-04T09:37:00,OLE Visit 5 - Month 3
4,ZX008-1503,ZX008-1501-0103-11,F,5.0,V8 PROLACTIN IS 94.9 H (INCREASED),Blood prolactin increased,2018-02-08,DOSE NOT CHANGED,N,MILD,25.72,RECOVERED/RESOLVED,PROLCTN,224.7,mIU/L,44.5,375.2,2017-11-07T08:10:00,OLE Visit 7 - Month 9
5,ZX008-1503,ZX008-1501-0103-11,F,5.0,V8 PROLACTIN IS 94.9 H (INCREASED),Blood prolactin increased,2018-02-08,DOSE NOT CHANGED,N,MILD,25.72,RECOVERED/RESOLVED,PROLCTN,2011.9,mIU/L,44.5,375.2,2018-02-08T08:45:00,OLE Visit 8 - Month 12
6,ZX008-1503,ZX008-1501-0118-04,F,2.0,ELEVATED PROLACTIN LEVEL,Blood prolactin increased,2018-01-25,DOSE NOT CHANGED,N,MILD,15.29,NOT RECOVERED/NOT RESOLVED,PROLCTN,911.6,mIU/L,44.5,375.2,2017-04-13T09:20:00,Visit 12 (Clinic)
7,ZX008-1503,ZX008-1501-0118-04,F,2.0,ELEVATED PROLACTIN LEVEL,Blood prolactin increased,2018-01-25,DOSE NOT CHANGED,N,MILD,15.29,NOT RECOVERED/NOT RESOLVED,PROLCTN,419.8,mIU/L,44.5,375.2,2017-05-31T11:30:00,OLE Visit 3 - Month 1
8,ZX008-1503,ZX008-1501-0118-04,F,2.0,ELEVATED PROLACTIN LEVEL,Blood prolactin increased,2018-01-25,DOSE NOT CHANGED,N,MILD,15.29,NOT RECOVERED/NOT RESOLVED,PROLCTN,214.1,mIU/L,44.5,375.2,2017-06-29T11:24:00,OLE Visit 4 - Month 2
9,ZX008-1503,ZX008-1501-0118-04,F,2.0,ELEVATED PROLACTIN LEVEL,Blood prolactin increased,2018-01-25,DOSE NOT CHANGED,N,MILD,15.29,NOT RECOVERED/NOT RESOLVED,PROLCTN,653.0,mIU/L,44.5,375.2,2017-07-31T11:53:00,OLE Visit 5 - Month 3


In [293]:
## Keep Lab dates which happen before AESI
aesi_t3_lb2 = aesi_t3_lb1.copy() 
aesi_t3_lb2['LBDT'] = aesi_t3_lb2.LBDTC.str[:10] ## only YYYY-MM-DD

aesi_t3_lb3 = aesi_t3_lb2[aesi_t3_lb2.LBDT <= aesi_t3_lb2.AESTDTC] ## LBDT <= AESTDTC
print(len(aesi_t3_lb3))
#print(aesi_t3_lb3)

12


In [294]:
## Select last lab data before AESI date

## reset index 
aesi_t3_lb4 = aesi_t3_lb3.reset_index()
## Sort data 
aesi_t3_lb5 = aesi_t3_lb4.sort_values(by=['USUBJID','LBDTC'])

## copy to new data frame
aesi_t3_lb6 = aesi_t3_lb5.copy() 

## Set up initial variablest
_usubjid = aesi_t3_lb6.loc[0,'USUBJID'] # first usubjid
aesi_t3_lb_last_index = aesi_t3_lb6.index[-1] ## last index number 

## use for-loop to derive days of exposure
for i, _df6 in aesi_t3_lb6.iterrows():
    
    #Last observation of each group. 
    if i > 0:
        if _df6['USUBJID'] != _usubjid:
            aesi_t3_lb6.loc[i-1, 'LAST_OBS'] = 1
            _usubjid = _df6['USUBJID']
       
    if i == (aesi_t3_lb_last_index):  # for last record  
        aesi_t3_lb6.loc[i, 'LAST_OBS'] = 1

## Kepp the last lab before 
aesi_t3_lb7 = aesi_t3_lb6[aesi_t3_lb6.LAST_OBS == 1]
aesi_t3_lb = aesi_t3_lb7.drop(columns=['LAST_OBS','index'])
print(aesi_t3_lb)

       STUDYID             USUBJID SEX  AGE  \
5   ZX008-1503  ZX008-1501-0103-11   F  5.0   
11  ZX008-1503  ZX008-1501-0118-04   F  2.0   

                                AETERM                    AEDECOD     AESTDTC  \
5   V8 PROLACTIN IS 94.9 H (INCREASED)  Blood prolactin increased  2018-02-08   
11            ELEVATED PROLACTIN LEVEL  Blood prolactin increased  2018-01-25   

               AEACN AESER AESEV  WEIGHTB                       AEOUT  \
5   DOSE NOT CHANGED     N  MILD    25.72          RECOVERED/RESOLVED   
11  DOSE NOT CHANGED     N  MILD    15.29  NOT RECOVERED/NOT RESOLVED   

   LBTESTCD LBSTRESC LBSTRESU  LBSTNRLO  LBSTNRHI                LBDTC  \
5   PROLCTN   2011.9    mIU/L      44.5     375.2  2018-02-08T08:45:00   
11  PROLCTN    913.7    mIU/L      44.5     375.2  2018-01-25T11:30:00   

                     VISIT        LBDT  
5   OLE Visit 8 - Month 12  2018-02-08  
11   OLE Visit 7 - Month 9  2018-01-25  


### Add SDTM EX last dose to Table 3 AESI 

In [295]:
## merge with Exposure EX
aesi_t3_lb_ex1 = pd.merge(aesi_t3_lb, _ex, how='left', on='USUBJID')
print(len(aesi_t3_lb_ex1))
#print(aesi_t3_lb_ex1)

4


In [296]:
## Keep exposure which happen before Lab
aesi_t3_lb_ex2 = aesi_t3_lb_ex1[aesi_t3_lb_ex1.EXSTDTC < aesi_t3_lb_ex1.LBDT] ## EXSTDTC < LBDT
print(len(aesi_t3_lb_ex2))
print(aesi_t3_lb_ex2)

4
      STUDYID             USUBJID SEX  AGE  \
0  ZX008-1503  ZX008-1501-0103-11   F  5.0   
1  ZX008-1503  ZX008-1501-0103-11   F  5.0   
2  ZX008-1503  ZX008-1501-0103-11   F  5.0   
3  ZX008-1503  ZX008-1501-0118-04   F  2.0   

                               AETERM                    AEDECOD     AESTDTC  \
0  V8 PROLACTIN IS 94.9 H (INCREASED)  Blood prolactin increased  2018-02-08   
1  V8 PROLACTIN IS 94.9 H (INCREASED)  Blood prolactin increased  2018-02-08   
2  V8 PROLACTIN IS 94.9 H (INCREASED)  Blood prolactin increased  2018-02-08   
3            ELEVATED PROLACTIN LEVEL  Blood prolactin increased  2018-01-25   

              AEACN AESER AESEV     ...      LBSTRESC LBSTRESU LBSTNRLO  \
0  DOSE NOT CHANGED     N  MILD     ...        2011.9    mIU/L     44.5   
1  DOSE NOT CHANGED     N  MILD     ...        2011.9    mIU/L     44.5   
2  DOSE NOT CHANGED     N  MILD     ...        2011.9    mIU/L     44.5   
3  DOSE NOT CHANGED     N  MILD     ...         913.7    mIU/L    

In [297]:
## Select last dose data before LAB date

## reset index 
aesi_t3_lb_ex2_reset = aesi_t3_lb_ex2.reset_index()
## Sort data 
aesi_t3_lb_ex3 = aesi_t3_lb_ex2_reset.sort_values(by=['USUBJID','LBDTC'])

## copy to new data frame
aesi_t3_lb_ex4 = aesi_t3_lb_ex3.copy() 

## Set up initial variablest
_usubjid = aesi_t3_lb_ex4.loc[0,'USUBJID'] # first usubjid
last_index = aesi_t3_lb_ex4.index[-1] ## last index number 

## use for-loop to derive days of exposure
for i, _df7 in aesi_t3_lb_ex4.iterrows():
    
    #Last observation of each group. 
    if i > 0:
        if _df7['USUBJID'] != _usubjid:
            aesi_t3_lb_ex4.loc[i-1, 'LAST_OBS'] = 1
            _usubjid = _df7['USUBJID']
       
    if i == (last_index):  # for last record  
        aesi_t3_lb_ex4.loc[i, 'LAST_OBS'] = 1

## Kepp the last lab before 
aesi_t3_lb_ex5 = aesi_t3_lb_ex4[aesi_t3_lb_ex4.LAST_OBS == 1]
aesi_t3_lb_ex = aesi_t3_lb_ex5.drop(columns=['LAST_OBS','index'])
print(aesi_t3_lb_ex)

      STUDYID             USUBJID SEX  AGE  \
2  ZX008-1503  ZX008-1501-0103-11   F  5.0   
3  ZX008-1503  ZX008-1501-0118-04   F  2.0   

                               AETERM                    AEDECOD     AESTDTC  \
2  V8 PROLACTIN IS 94.9 H (INCREASED)  Blood prolactin increased  2018-02-08   
3            ELEVATED PROLACTIN LEVEL  Blood prolactin increased  2018-01-25   

              AEACN AESER AESEV     ...      LBSTRESC LBSTRESU LBSTNRLO  \
2  DOSE NOT CHANGED     N  MILD     ...        2011.9    mIU/L     44.5   
3  DOSE NOT CHANGED     N  MILD     ...         913.7    mIU/L     44.5   

  LBSTNRHI                LBDTC                   VISIT        LBDT EXDOSTXT  \
2    375.2  2018-02-08T08:45:00  OLE Visit 8 - Month 12  2018-02-08      0.8   
3    375.2  2018-01-25T11:30:00   OLE Visit 7 - Month 9  2018-01-25      0.2   

      EXSTDTC     EXENDTC  
2  2017-04-06  2018-06-17  
3  2017-05-02  2018-06-16  

[2 rows x 23 columns]


### Find Seizures which happens within 48 hours prior to lab data

In [298]:
## Add seizures data to Table 3 AESI 
aesi_t3_lb_ex_xs1 = pd.merge(aesi_t3_lb_ex, _xs, how='left', on='USUBJID')
print(len(aesi_t3_lb_ex_xs1))
print(aesi_t3_lb_ex_xs1[['USUBJID','AESTDTC','LBDT','EXSTDTC','XSORRES','XSSTDTC']].tail())

2509
                 USUBJID     AESTDTC        LBDT     EXSTDTC  \
2504  ZX008-1501-0118-04  2018-01-25  2018-01-25  2017-05-02   
2505  ZX008-1501-0118-04  2018-01-25  2018-01-25  2017-05-02   
2506  ZX008-1501-0118-04  2018-01-25  2018-01-25  2017-05-02   
2507  ZX008-1501-0118-04  2018-01-25  2018-01-25  2017-05-02   
2508  ZX008-1501-0118-04  2018-01-25  2018-01-25  2017-05-02   

                         XSORRES     XSSTDTC  
2504  Evening (6:00pm - 11:59pm)  2017-01-07  
2505  Evening (6:00pm - 11:59pm)  2017-02-08  
2506  Evening (6:00pm - 11:59pm)  2017-03-12  
2507  Morning (8:00am - 11:59am)  2017-02-25  
2508  Morning (8:00am - 11:59am)  2017-12-11  


In [299]:
## Assign blank for missing data
#aesi_t3_lb_ex_xs1['XSORRES'] = aesi_t3_lb_ex_xs1['XSORRES'].fillna(" ")
#aesi_t3_lb_ex_xs1['XSSTDTC'] = aesi_t3_lb_ex_xs1['XSSTDTC'].fillna(" ")
#print(aesi_t3_lb_ex_xs1[['USUBJID','AESTDTC','LBDT','EXSTDTC','XSORRES','XSSTDTC']].tail())

In [300]:
print(aesi_t3_lb_ex_xs1.columns.tolist())

['STUDYID', 'USUBJID', 'SEX', 'AGE', 'AETERM', 'AEDECOD', 'AESTDTC', 'AEACN', 'AESER', 'AESEV', 'WEIGHTB', 'AEOUT', 'LBTESTCD', 'LBSTRESC', 'LBSTRESU', 'LBSTNRLO', 'LBSTNRHI', 'LBDTC', 'VISIT', 'LBDT', 'EXDOSTXT', 'EXSTDTC', 'EXENDTC', 'XSTESTCD', 'XSTEST', 'XSORRES', 'XSDTC', 'XSSTDTC']


In [301]:
## Select Seizure data prior to LBDT
aesi_t3_lb_ex_xs2 = aesi_t3_lb_ex_xs1[aesi_t3_lb_ex_xs1.LBDT >= aesi_t3_lb_ex_xs1.XSSTDTC] 
print(len(aesi_t3_lb_ex_xs2))

## Select last seizure data prior to LAB data

## Sort data 
aesi_t3_lb_ex_xs3 = aesi_t3_lb_ex_xs2.sort_values(by=['USUBJID','LBDTC','XSSTDTC'])
## reset index 
aesi_t3_lb_ex_xs4 = aesi_t3_lb_ex_xs3.reset_index()

## copy to new data frame
aesi_t3_lb_ex_xs5 = aesi_t3_lb_ex_xs4.copy() 

## Set up initial variablest
_usubjid = aesi_t3_lb_ex_xs5.loc[0,'USUBJID'] # first usubjid
last_index = aesi_t3_lb_ex_xs5.index[-1] ## last index number 

## use for-loop to find the last seizure data
for i, _df8 in aesi_t3_lb_ex_xs5.iterrows():
    
    #Last observation of each group. 
    if i > 0:
        if _df8['USUBJID'] != _usubjid:
            aesi_t3_lb_ex_xs5.loc[i-1, 'LAST_OBS'] = 1
            _usubjid = _df8['USUBJID']
       
    if i == (last_index):  # for last record  
        aesi_t3_lb_ex_xs5.loc[i, 'LAST_OBS'] = 1
    
    # Day difference between Seizure and Lab date
    if len(_df8.LBDT) == 8 and len(_df8.XSSTDTC) == 8 :
        aesi_t3_lb_ex_xs5.loc[i, 'DAY_XS_LB'] = (dt.datetime.strptime(_df8.LBDT,'%Y-%m-%d') - 
                                             dt.datetime.strptime(_df8.XSSTDTC,'%Y-%m-%d')).days
    else:
        #aesi_t3_lb_ex_xs5.loc[i, 'DAY_XS_LB'] = None
        aesi_t3_lb_ex_xs5.loc[i, 'DAY_XS_LB'] = (dt.datetime.strptime(_df8.LBDT,'%Y-%m-%d') - 
                                             dt.datetime.strptime(_df8.XSSTDTC,'%Y-%m-%d')).days
    
    # Derive Days of differnet doses and Days of exposures from LB date
    if _df8.XSSTDTC > _df8.EXENDTC:
        aesi_t3_lb_ex_xs5.loc[i, 'LB_EXSTDY'] = (dt.datetime.strptime(_df8.EXENDTC, '%Y-%m-%d') - 
                                                 dt.datetime.strptime(_df8.EXSTDTC, '%Y-%m-%d')).days + 1 
    else:
        aesi_t3_lb_ex_xs5.loc[i, 'LB_EXSTDY'] = (dt.datetime.strptime(_df8.LBDT, '%Y-%m-%d') - 
                                                 dt.datetime.strptime(_df8.EXSTDTC, '%Y-%m-%d')).days 

## Kepp the last seizure prior to lab data
aesi_t3_lb_ex_xs6 = aesi_t3_lb_ex_xs5[aesi_t3_lb_ex_xs5.LAST_OBS == 1]

## reset index 
aesi_t3_lb_ex_xs7 = aesi_t3_lb_ex_xs6.reset_index()

aesi_t3_lb_ex_xs = aesi_t3_lb_ex_xs7.drop(columns=['LAST_OBS','index','level_0'])
print(aesi_t3_lb_ex_xs[['USUBJID','EXSTDTC','LBDT','XSSTDTC','DAY_XS_LB','LB_EXSTDY']])

2178
              USUBJID     EXSTDTC        LBDT     XSSTDTC  DAY_XS_LB  \
0  ZX008-1501-0103-11  2017-04-06  2018-02-08  2018-02-08        0.0   
1  ZX008-1501-0118-04  2017-05-02  2018-01-25  2017-12-11       45.0   

   LB_EXSTDY  
0      308.0  
1      268.0  


In [302]:
print(aesi_t3_lb_ex_xs.columns.tolist())

['STUDYID', 'USUBJID', 'SEX', 'AGE', 'AETERM', 'AEDECOD', 'AESTDTC', 'AEACN', 'AESER', 'AESEV', 'WEIGHTB', 'AEOUT', 'LBTESTCD', 'LBSTRESC', 'LBSTRESU', 'LBSTNRLO', 'LBSTNRHI', 'LBDTC', 'VISIT', 'LBDT', 'EXDOSTXT', 'EXSTDTC', 'EXENDTC', 'XSTESTCD', 'XSTEST', 'XSORRES', 'XSDTC', 'XSSTDTC', 'DAY_XS_LB', 'LB_EXSTDY']


In [303]:
## Final data set for Table 3
aesi_t3_final = aesi_t3_lb_ex_xs[['STUDYID', 'USUBJID', 'SEX', 'AGE', 'WEIGHTB','AETERM','AEDECOD', 'AESTDTC', 'AEACN', 
                                  'AESER', 'AESEV',  'AEOUT', 
                                  'LBTESTCD', 'LBSTRESC', 'LBSTRESU', 'LBSTNRLO', 'LBSTNRHI', 'LBDTC', 'VISIT', 'LBDT', 
                                  'EXDOSTXT', 'EXSTDTC', 'EXENDTC', 
                                  'XSTESTCD', 'XSTEST', 'XSORRES', 'XSDTC', 'XSSTDTC', 'DAY_XS_LB', 'LB_EXSTDY']]

## Table 4

### Prepare SDTM VS datasets

In [304]:
### Read SDTM VS datasets
with open('../sdtm/vs.xpt', 'rb') as _vs1: 
    _vs2 = xport.to_columns(_vs1)
    _vs3 = pd.DataFrame(_vs2)

In [305]:
_vs4 = _vs3[(_vs3.VSTESTCD == 'DIABP') | (_vs3.VSTESTCD == 'SYSBP') | (_vs3.VSTESTCD == 'HR')]
_vs = _vs4[['USUBJID','VSTESTCD','VSSTRESC','VSSTRESU','VISIT','VISITNUM','VSDTC']]
print(len(_vs))

4738


In [306]:
## Create SYSBP / DIABP  
advs_diabp2 = _vs[(_vs3.VSTESTCD == 'DIABP')]
advs_diabp = advs_diabp2.rename(columns={"VSSTRESC":"DIABP"})
advs_diabp['VSTESTCD'] = 'SYSBP / DIABP'

advs_sysbp2 = _vs[(_vs3.VSTESTCD == 'SYSBP')]
advs_sysbp = advs_sysbp2.rename(columns={"VSSTRESC":"SYSBP"})
advs_sysbp['VSTESTCD'] = 'SYSBP / DIABP'

advs2 = pd.merge(advs_diabp, advs_sysbp, how='inner', on=['USUBJID','VSTESTCD','VSSTRESU','VISITNUM','VISIT','VSDTC'])
advs2['VSSTRESC'] = advs2.SYSBP + ' / ' + advs2.DIABP
advs3 = advs2[['USUBJID','VSTESTCD','VSSTRESC','VSSTRESU','VISITNUM','VISIT','VSDTC']]
print(len(advs_diabp), len(advs_sysbp), len(advs3))
advs3.head()

1577 1577 1577


C:\Users\klee\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\klee\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,USUBJID,VSTESTCD,VSSTRESC,VSSTRESU,VISITNUM,VISIT,VSDTC
0,ZX008-1501-0101-01,SYSBP / DIABP,101 / 69,mmHg,1001.0,OLE Visit 1 - Day 1,2016-09-28
1,ZX008-1501-0101-01,SYSBP / DIABP,92 / 54,mmHg,1003.0,OLE Visit 3 - Month 1,2016-10-27
2,ZX008-1501-0101-01,SYSBP / DIABP,90 / 70,mmHg,1004.0,OLE Visit 4 - Month 2,2016-11-29
3,ZX008-1501-0101-01,SYSBP / DIABP,102 / 83,mmHg,1005.0,OLE Visit 5 - Month 3,2017-01-04
4,ZX008-1501-0101-01,SYSBP / DIABP,103 / 67,mmHg,1006.0,OLE Visit 6 - Month 6,2017-03-28


In [307]:
## Concatenate 'SYSBP / DIABP'
advs4 = pd.concat([_vs, advs3])
advs = advs4.sort_values(by=['USUBJID','VSTESTCD','VISITNUM','VISIT','VSDTC'])
print(len(advs))
print(advs[:5])

6315
               USUBJID                  VISIT  VISITNUM       VSDTC VSSTRESC  \
11  ZX008-1501-0101-01    OLE Visit 1 - Day 1    1001.0  2016-09-28       69   
12  ZX008-1501-0101-01  OLE Visit 3 - Month 1    1003.0  2016-10-27       54   
13  ZX008-1501-0101-01  OLE Visit 4 - Month 2    1004.0  2016-11-29       70   
14  ZX008-1501-0101-01  OLE Visit 5 - Month 3    1005.0  2017-01-04       83   
15  ZX008-1501-0101-01  OLE Visit 6 - Month 6    1006.0  2017-03-28       67   

   VSSTRESU VSTESTCD  
11     mmHg    DIABP  
12     mmHg    DIABP  
13     mmHg    DIABP  
14     mmHg    DIABP  
15     mmHg    DIABP  


### Prepare Table 4 AESI

In [308]:
adae_sl['AEDECOD'].value_counts()

Echocardiogram abnormal               34
Blood glucose decreased               14
Blood pressure diastolic increased     7
Blood pressure increased               6
Heart rate increased                   3
Hypoglycaemia                          2
Tachycardia                            2
Blood prolactin increased              2
Blood pressure systolic increased      1
Dysphagia                              1
Mitral valve incompetence              1
Cardiac murmur                         1
Name: AEDECOD, dtype: int64

In [309]:
## Subsetting AESI for below AE PTs
aesi_t4_1 = adae_sl[(adae_sl.AEDECOD == 'Blood pressure diastolic increased') 
                    | (adae_sl.AEDECOD == 'Blood pressure increased') 
                    | (adae_sl.AEDECOD == 'Blood pressure systolic increased') 
                    | (adae_sl.AEDECOD == 'Heart rate increased') 
                    | (adae_sl.AEDECOD == 'Cardiac murmur') 
                    | (adae_sl.AEDECOD == 'Tachycardia') 
                    | (adae_sl.AEDECOD == 'Diastolic hypertension') 
                    | (adae_sl.AEDECOD == 'Hypertension') 
                    | (adae_sl.AEDECOD == 'Pneumonia') 
                    | (adae_sl.AEDECOD == 'Dysphagia')]

aesi_t4_2 = aesi_t4_1.reset_index()
aesi_t4 = aesi_t4_2[['STUDYID','USUBJID','SEX','AGE','AETERM','AEDECOD','AESTDTC','AEACN','AESER','AESEV','WEIGHTB','AEOUT']]
print(len(aesi_t4))
aesi_t4

21


,STUDYID,USUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,AEACN,AESER,AESEV,WEIGHTB,AEOUT
0,ZX008-1503,ZX008-1501-0101-01,F,4.0,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04,DOSE NOT CHANGED,N,MILD,17.69,RECOVERED/RESOLVED
1,ZX008-1503,ZX008-1501-0103-03,F,8.0,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08,DOSE NOT CHANGED,N,MILD,29.21,NOT RECOVERED/NOT RESOLVED
2,ZX008-1503,ZX008-1501-0103-12,F,7.0,V6 DIASTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure diastolic increased,2017-08-09,DOSE NOT CHANGED,N,MILD,23.41,RECOVERED/RESOLVED
3,ZX008-1503,ZX008-1501-0103-12,F,7.0,V6 SYSTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure systolic increased,2017-08-09,DOSE NOT CHANGED,N,MILD,23.41,RECOVERED/RESOLVED
4,ZX008-1503,ZX008-1501-0103-12,F,7.0,V8 DIASTOLIC BLOOD PRESSURE INCREASE MORE THAN...,Blood pressure diastolic increased,2018-02-06,DOSE NOT CHANGED,N,MILD,23.41,NOT RECOVERED/NOT RESOLVED
5,ZX008-1503,ZX008-1501-0103-16,M,3.0,V4 BP INCREASED >30% FROM SCREENING BP,Blood pressure increased,2017-07-27,DOSE NOT CHANGED,N,MILD,22.82,RECOVERED/RESOLVED
6,ZX008-1503,ZX008-1501-0103-16,M,3.0,V4 HR INCREASED >30% FROM SCREENING HR,Heart rate increased,2017-07-27,DOSE NOT CHANGED,N,MILD,22.82,NOT RECOVERED/NOT RESOLVED
7,ZX008-1503,ZX008-1501-0110-11,M,4.0,INCREASED HEART RATE BY MORE THAN 30% FROM SCR...,Heart rate increased,2018-02-19,DOSE NOT CHANGED,N,MILD,14.79,NOT RECOVERED/NOT RESOLVED
8,ZX008-1503,ZX008-1501-0115-03,M,7.0,DYSPHAGIA,Dysphagia,2018-03-07,DOSE NOT CHANGED,Y,SEVERE,21.82,RECOVERED/RESOLVED
9,ZX008-1503,ZX008-1501-0119-09,F,4.0,ELEVATED BLOOD PRESSURE GREATER THAN 30% FROM ...,Blood pressure increased,2018-02-15,DOSE NOT CHANGED,N,MILD,16.60,RECOVERED/RESOLVED


In [310]:
## Print to local drive
t4 = open('./out/aesi_t4_init.rtf', 'w')
print(aesi_t4[['USUBJID','AETERM','AEDECOD','AESTDTC','AEACN','AESER','AESEV',]].to_string(), file=t4)
t4.close()

### Add SDTM EX last dose to Table 4 AESI 

In [311]:
## merge with Exposure EX
aesi_t4_ex1 = pd.merge(aesi_t4, _ex, how='left', on='USUBJID')
print(len(aesi_t4_ex1))
aesi_t4_ex1

65


,STUDYID,USUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,AEACN,AESER,AESEV,WEIGHTB,AEOUT,EXDOSTXT,EXSTDTC,EXENDTC
0,ZX008-1503,ZX008-1501-0101-01,F,4.0,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04,DOSE NOT CHANGED,N,MILD,17.69,RECOVERED/RESOLVED,0.2,2016-09-28,2016-10-26
1,ZX008-1503,ZX008-1501-0101-01,F,4.0,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04,DOSE NOT CHANGED,N,MILD,17.69,RECOVERED/RESOLVED,0.4,2016-10-27,2016-11-28
2,ZX008-1503,ZX008-1501-0101-01,F,4.0,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04,DOSE NOT CHANGED,N,MILD,17.69,RECOVERED/RESOLVED,0.6,2016-11-29,2017-06-22
3,ZX008-1503,ZX008-1501-0101-01,F,4.0,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04,DOSE NOT CHANGED,N,MILD,17.69,RECOVERED/RESOLVED,0.8,2017-06-23,2018-06-17
4,ZX008-1503,ZX008-1501-0103-03,F,8.0,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08,DOSE NOT CHANGED,N,MILD,29.21,NOT RECOVERED/NOT RESOLVED,0.2,2016-11-08,2016-12-07
5,ZX008-1503,ZX008-1501-0103-03,F,8.0,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08,DOSE NOT CHANGED,N,MILD,29.21,NOT RECOVERED/NOT RESOLVED,0.4,2016-12-08,2017-01-09
6,ZX008-1503,ZX008-1501-0103-03,F,8.0,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08,DOSE NOT CHANGED,N,MILD,29.21,NOT RECOVERED/NOT RESOLVED,0.6,2017-01-10,2018-06-15
7,ZX008-1503,ZX008-1501-0103-03,F,8.0,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08,DOSE NOT CHANGED,N,MILD,29.21,NOT RECOVERED/NOT RESOLVED,0.8,2017-02-09,2018-02-04
8,ZX008-1503,ZX008-1501-0103-12,F,7.0,V6 DIASTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure diastolic increased,2017-08-09,DOSE NOT CHANGED,N,MILD,23.41,RECOVERED/RESOLVED,0.2,2017-02-10,2017-03-09
9,ZX008-1503,ZX008-1501-0103-12,F,7.0,V6 DIASTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure diastolic increased,2017-08-09,DOSE NOT CHANGED,N,MILD,23.41,RECOVERED/RESOLVED,0.6,2017-03-10,2017-04-13


In [312]:
## Keep exposure which happen before AE
aesi_t4_ex2 = aesi_t4_ex1[aesi_t4_ex1.EXSTDTC < aesi_t4_ex1.AESTDTC] ## EXSTDTC <= AESTDTC
print(len(aesi_t4_ex2))
aesi_t4_ex2

43


,STUDYID,USUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,AEACN,AESER,AESEV,WEIGHTB,AEOUT,EXDOSTXT,EXSTDTC,EXENDTC
0,ZX008-1503,ZX008-1501-0101-01,F,4.0,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04,DOSE NOT CHANGED,N,MILD,17.69,RECOVERED/RESOLVED,0.2,2016-09-28,2016-10-26
1,ZX008-1503,ZX008-1501-0101-01,F,4.0,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04,DOSE NOT CHANGED,N,MILD,17.69,RECOVERED/RESOLVED,0.4,2016-10-27,2016-11-28
2,ZX008-1503,ZX008-1501-0101-01,F,4.0,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04,DOSE NOT CHANGED,N,MILD,17.69,RECOVERED/RESOLVED,0.6,2016-11-29,2017-06-22
4,ZX008-1503,ZX008-1501-0103-03,F,8.0,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08,DOSE NOT CHANGED,N,MILD,29.21,NOT RECOVERED/NOT RESOLVED,0.2,2016-11-08,2016-12-07
5,ZX008-1503,ZX008-1501-0103-03,F,8.0,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08,DOSE NOT CHANGED,N,MILD,29.21,NOT RECOVERED/NOT RESOLVED,0.4,2016-12-08,2017-01-09
6,ZX008-1503,ZX008-1501-0103-03,F,8.0,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08,DOSE NOT CHANGED,N,MILD,29.21,NOT RECOVERED/NOT RESOLVED,0.6,2017-01-10,2018-06-15
7,ZX008-1503,ZX008-1501-0103-03,F,8.0,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08,DOSE NOT CHANGED,N,MILD,29.21,NOT RECOVERED/NOT RESOLVED,0.8,2017-02-09,2018-02-04
8,ZX008-1503,ZX008-1501-0103-12,F,7.0,V6 DIASTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure diastolic increased,2017-08-09,DOSE NOT CHANGED,N,MILD,23.41,RECOVERED/RESOLVED,0.2,2017-02-10,2017-03-09
9,ZX008-1503,ZX008-1501-0103-12,F,7.0,V6 DIASTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure diastolic increased,2017-08-09,DOSE NOT CHANGED,N,MILD,23.41,RECOVERED/RESOLVED,0.6,2017-03-10,2017-04-13
10,ZX008-1503,ZX008-1501-0103-12,F,7.0,V6 DIASTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure diastolic increased,2017-08-09,DOSE NOT CHANGED,N,MILD,23.41,RECOVERED/RESOLVED,0.8,2017-04-14,2018-06-18


In [313]:
## Select last dose data before AESTDTC date

## Sort data 
aesi_t4_ex2_sort = aesi_t4_ex2.sort_values(by=['USUBJID','AETERM','AEDECOD','AESTDTC'])

## reset index 
aesi_t4_ex3 = aesi_t4_ex2_sort.reset_index()

## copy to new data frame
aesi_t4_ex4 = aesi_t4_ex3.copy() 

## Set up initial variables
_usubjid = aesi_t4_ex4.loc[0,'USUBJID'] # first usubjid
_aedecod = aesi_t4_ex4.loc[0,'AEDECOD'] # first usubjid
_aestdtc = aesi_t4_ex4.loc[0,'AESTDTC'] # first AE start date
last_index = aesi_t4_ex4.index[-1] ## last index number 

## use for-loop to derive last day of exposure
for i, _df9 in aesi_t4_ex4.iterrows():
    
    #Last observation of each group - USUBJID & AEDECOD & AESTDTC 
    if i > 0:
        if _df9['USUBJID'] != _usubjid:
            aesi_t4_ex4.loc[i-1, 'LAST_OBS'] = 1
            _usubjid = _df9['USUBJID']
        if _df9['AEDECOD'] != _aedecod:
            aesi_t4_ex4.loc[i-1, 'LAST_OBS'] = 1
            _aedecod = _df9['AEDECOD']
        if _df9['AESTDTC'] != _aestdtc:
            aesi_t4_ex4.loc[i-1, 'LAST_OBS'] = 1
            _aestdtc = _df9['AESTDTC']
    if i == (last_index):  # for last record  
        aesi_t4_ex4.loc[i, 'LAST_OBS'] = 1

aesi_t4_ex4[['USUBJID','AETERM','AEDECOD','AESTDTC']]

,USUBJID,AETERM,AEDECOD,AESTDTC
0,ZX008-1501-0101-01,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04
1,ZX008-1501-0101-01,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04
2,ZX008-1501-0101-01,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04
3,ZX008-1501-0103-03,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08
4,ZX008-1501-0103-03,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08
5,ZX008-1501-0103-03,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08
6,ZX008-1501-0103-03,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08
7,ZX008-1501-0103-12,V6 DIASTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure diastolic increased,2017-08-09
8,ZX008-1501-0103-12,V6 DIASTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure diastolic increased,2017-08-09
9,ZX008-1501-0103-12,V6 DIASTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure diastolic increased,2017-08-09


In [314]:
## Kepp the last EX bfore AE
aesi_t4_ex5 = aesi_t4_ex4[aesi_t4_ex4.LAST_OBS == 1]
aesi_t4_ex6 = aesi_t4_ex5[['USUBJID','AETERM','AEDECOD','AESTDTC','EXDOSTXT','EXSTDTC','EXENDTC']]
print(len(aesi_t4_ex6))
print(aesi_t4_ex6)

20
                  USUBJID                                             AETERM  \
2      ZX008-1501-0101-01  1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...   
6      ZX008-1501-0103-03                 V7 BP INCRASED >30% FROM SCREENING   
9      ZX008-1501-0103-12      V6 DIASTOLIC BP INCREASED >30% FROM SCREENING   
12     ZX008-1501-0103-12       V6 SYSTOLIC BP INCREASED >30% FROM SCREENING   
15     ZX008-1501-0103-12  V8 DIASTOLIC BLOOD PRESSURE INCREASE MORE THAN...   
17     ZX008-1501-0103-16             V4 BP INCREASED >30% FROM SCREENING BP   
19     ZX008-1501-0103-16             V4 HR INCREASED >30% FROM SCREENING HR   
22     ZX008-1501-0110-11  INCREASED HEART RATE BY MORE THAN 30% FROM SCR...   
25     ZX008-1501-0115-03                                          DYSPHAGIA   
26     ZX008-1501-0119-09  ELEVATED BLOOD PRESSURE GREATER THAN 30% FROM ...   
28     ZX008-1501-0127-01                                        TACHYCARDIA   
32     ZX008-1501-0201-07            

In [315]:
aesi_t4_ex7 = pd.merge(aesi_t4, aesi_t4_ex6, how='left', on=['USUBJID','AETERM','AEDECOD','AESTDTC'])
aesi_t4_ex7['EXSTDTC'] = aesi_t4_ex7['EXSTDTC'].fillna(" ")
aesi_t4_ex7['EXENDTC'] = aesi_t4_ex7['EXENDTC'].fillna(" ")
aesi_t4_ex7['EXDOSTXT'] = aesi_t4_ex7['EXDOSTXT'].fillna(" ")
print(len(aesi_t4_ex7))
aesi_t4_ex = aesi_t4_ex7.copy()
aesi_t4_ex

21


,STUDYID,USUBJID,SEX,AGE,AETERM,AEDECOD,AESTDTC,AEACN,AESER,AESEV,WEIGHTB,AEOUT,EXDOSTXT,EXSTDTC,EXENDTC
0,ZX008-1503,ZX008-1501-0101-01,F,4.0,"1/6 SOFT,VIBRATORY SYSTOLIC EJECTION MURMUR AT...",Cardiac murmur,2017-01-04,DOSE NOT CHANGED,N,MILD,17.69,RECOVERED/RESOLVED,0.6,2016-11-29,2017-06-22
1,ZX008-1503,ZX008-1501-0103-03,F,8.0,V7 BP INCRASED >30% FROM SCREENING,Blood pressure increased,2017-08-08,DOSE NOT CHANGED,N,MILD,29.21,NOT RECOVERED/NOT RESOLVED,0.8,2017-02-09,2018-02-04
2,ZX008-1503,ZX008-1501-0103-12,F,7.0,V6 DIASTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure diastolic increased,2017-08-09,DOSE NOT CHANGED,N,MILD,23.41,RECOVERED/RESOLVED,0.8,2017-04-14,2018-06-18
3,ZX008-1503,ZX008-1501-0103-12,F,7.0,V6 SYSTOLIC BP INCREASED >30% FROM SCREENING,Blood pressure systolic increased,2017-08-09,DOSE NOT CHANGED,N,MILD,23.41,RECOVERED/RESOLVED,0.8,2017-04-14,2018-06-18
4,ZX008-1503,ZX008-1501-0103-12,F,7.0,V8 DIASTOLIC BLOOD PRESSURE INCREASE MORE THAN...,Blood pressure diastolic increased,2018-02-06,DOSE NOT CHANGED,N,MILD,23.41,NOT RECOVERED/NOT RESOLVED,0.8,2017-04-14,2018-06-18
5,ZX008-1503,ZX008-1501-0103-16,M,3.0,V4 BP INCREASED >30% FROM SCREENING BP,Blood pressure increased,2017-07-27,DOSE NOT CHANGED,N,MILD,22.82,RECOVERED/RESOLVED,0.4,2017-06-22,2018-02-21
6,ZX008-1503,ZX008-1501-0103-16,M,3.0,V4 HR INCREASED >30% FROM SCREENING HR,Heart rate increased,2017-07-27,DOSE NOT CHANGED,N,MILD,22.82,NOT RECOVERED/NOT RESOLVED,0.4,2017-06-22,2018-02-21
7,ZX008-1503,ZX008-1501-0110-11,M,4.0,INCREASED HEART RATE BY MORE THAN 30% FROM SCR...,Heart rate increased,2018-02-19,DOSE NOT CHANGED,N,MILD,14.79,NOT RECOVERED/NOT RESOLVED,0.6,2017-10-16,2018-02-18
8,ZX008-1503,ZX008-1501-0115-03,M,7.0,DYSPHAGIA,Dysphagia,2018-03-07,DOSE NOT CHANGED,Y,SEVERE,21.82,RECOVERED/RESOLVED,0.6,2017-12-20,2018-05-28
9,ZX008-1503,ZX008-1501-0119-09,F,4.0,ELEVATED BLOOD PRESSURE GREATER THAN 30% FROM ...,Blood pressure increased,2018-02-15,DOSE NOT CHANGED,N,MILD,16.60,RECOVERED/RESOLVED,0.2,2018-01-17,2018-02-14


### Merge approriate SDTM VS to Table 4 AESI

In [316]:
aesi_t4_ex['AEDECOD'].value_counts()

Blood pressure diastolic increased    7
Blood pressure increased              6
Heart rate increased                  3
Tachycardia                           2
Blood pressure systolic increased     1
Dysphagia                             1
Cardiac murmur                        1
Name: AEDECOD, dtype: int64

In [317]:
_vs['VSTESTCD'].value_counts()

HR       1584
DIABP    1577
SYSBP    1577
Name: VSTESTCD, dtype: int64

In [318]:
## Assign appropriate Vital sign from AE PTs
## Sort data 
aesi_t4_ex_sort = aesi_t4_ex.sort_values(by=['USUBJID','AEDECOD','AESTDTC'])

## reset index 
aesi_t4_ex_vs1 = aesi_t4_ex_sort.reset_index()

## copy to new data frame
aesi_t4_ex_vs2 = aesi_t4_ex_vs1.copy() ## use for-loop to derive days of exposure
for i, _df10 in aesi_t4_ex_vs2 .iterrows():
    
    #Last observation of each group - USUBJID & AESTDTC 
    if _df10['AEDECOD'] == 'Blood pressure diastolic increased':
        aesi_t4_ex_vs2.loc[i, 'VSTESTCD'] = 'DIABP'
    elif _df10['AEDECOD'] == 'Blood pressure systolic increased':
        aesi_t4_ex_vs2.loc[i, 'VSTESTCD'] = 'SYSBP'
    elif _df10['AEDECOD'] == 'Heart rate increased':
        aesi_t4_ex_vs2.loc[i, 'VSTESTCD'] = 'HR'
    elif _df10['AEDECOD'] == 'Blood pressure increased':
        aesi_t4_ex_vs2.loc[i, 'VSTESTCD'] = 'SYSBP / DIABP'
    elif _df10['AEDECOD'] == 'Hypertension':
        aesi_t4_ex_vs2.loc[i, 'VSTESTCD'] = 'SYSBP / DIABP'
    elif _df10['AEDECOD'] == 'Diastolic hypertension':
        aesi_t4_ex_vs2.loc[i, 'VSTESTCD'] = 'DIABP'
    else:
        aesi_t4_ex_vs2.loc[i, 'VSTESTCD'] = 'NA'  
print(len(aesi_t4_ex_vs2))
print(aesi_t4_ex_vs2[['USUBJID','AEDECOD','AESTDTC','VSTESTCD']].head())
aesi_t4_ex_vs2 = aesi_t4_ex_vs2.drop(columns=['index'])

21
              USUBJID                             AEDECOD     AESTDTC  \
0  ZX008-1501-0101-01                      Cardiac murmur  2017-01-04   
1  ZX008-1501-0103-03            Blood pressure increased  2017-08-08   
2  ZX008-1501-0103-12  Blood pressure diastolic increased  2017-08-09   
3  ZX008-1501-0103-12  Blood pressure diastolic increased  2018-02-06   
4  ZX008-1501-0103-12   Blood pressure systolic increased  2017-08-09   

        VSTESTCD  
0             NA  
1  SYSBP / DIABP  
2          DIABP  
3          DIABP  
4          SYSBP  


In [319]:
## Merge with Vital
aesi_t4_ex_vs3 = pd.merge(aesi_t4_ex_vs2, advs, how='left', on=['USUBJID','VSTESTCD'])
print(len(aesi_t4_ex_vs3))
aesi_t4_ex_vs3.groupby(['USUBJID','AESTDTC','VSTESTCD'])['VSSTRESC'].count()

133


USUBJID                AESTDTC     VSTESTCD     
ZX008-1501-0101-01     2017-01-04  NA                0
ZX008-1501-0103-03     2017-08-08  SYSBP / DIABP    11
ZX008-1501-0103-12     2017-08-09  DIABP             9
                                   SYSBP             9
                       2018-02-06  DIABP             9
ZX008-1501-0103-16     2017-07-27  HR                7
                                   SYSBP / DIABP     7
ZX008-1501-0110-11     2018-02-19  HR                7
ZX008-1501-0115-03     2018-03-07  NA                0
ZX008-1501-0119-09     2018-02-15  SYSBP / DIABP     5
ZX008-1501-0127-01     2018-03-14  NA                0
ZX008-1501-0201-07     2018-03-01  SYSBP / DIABP     6
ZX008-1502-0302-01     2017-02-20  DIABP             9
                       2017-03-20  HR                9
                       2017-05-22  DIABP             9
ZX008-1502-0701-11     2017-11-16  DIABP             7
ZX008-1502-0801-01     2017-02-17  NA                0
ZX008-1504-C1-14

In [320]:
## Keep vitals which happen before AESTDTC
aesi_t4_ex_vs4 = aesi_t4_ex_vs3[(aesi_t4_ex_vs3.VSDTC <= aesi_t4_ex_vs3.AESTDTC) | (aesi_t4_ex_vs3.VSTESTCD == 'NA')] ## EXSTDTC <=AESTDTC
aesi_t4_ex_vs4['VSDTC'] = aesi_t4_ex_vs4['VSDTC'].fillna(" ")
print(len(aesi_t4_ex_vs4))
print(aesi_t4_ex_vs4.columns.tolist())
print(aesi_t4_ex_vs4[['AESTDTC','VSDTC','EXSTDTC','EXENDTC']])

94
['STUDYID', 'USUBJID', 'SEX', 'AGE', 'AETERM', 'AEDECOD', 'AESTDTC', 'AEACN', 'AESER', 'AESEV', 'WEIGHTB', 'AEOUT', 'EXDOSTXT', 'EXSTDTC', 'EXENDTC', 'VSTESTCD', 'VISIT', 'VISITNUM', 'VSDTC', 'VSSTRESC', 'VSSTRESU']
        AESTDTC       VSDTC     EXSTDTC     EXENDTC
0    2017-01-04              2016-11-29  2017-06-22
1    2017-08-08  2016-11-08  2017-02-09  2018-02-04
2    2017-08-08  2016-11-22  2017-02-09  2018-02-04
3    2017-08-08  2016-12-08  2017-02-09  2018-02-04
4    2017-08-08  2017-01-10  2017-02-09  2018-02-04
5    2017-08-08  2017-02-09  2017-02-09  2018-02-04
6    2017-08-08  2017-05-09  2017-02-09  2018-02-04
7    2017-08-08  2017-08-08  2017-02-09  2018-02-04
10   2017-08-08              2017-02-09  2018-02-04
11   2017-08-08              2017-02-09  2018-02-04
12   2017-08-09  2017-02-10  2017-04-14  2018-06-18
13   2017-08-09  2017-03-10  2017-04-14  2018-06-18
14   2017-08-09  2017-04-14  2017-04-14  2018-06-18
15   2017-08-09  2017-05-15  2017-04-14  2018-06-18
1

C:\Users\klee\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [321]:
## Select last vitlas data before AESTDTC date

## Sort data 
aesi_t4_ex_vs5 = aesi_t4_ex_vs4.sort_values(by=['USUBJID','AEDECOD','AESTDTC','VSTESTCD','VSDTC'])

## reset index 
aesi_t4_ex_vs6 = aesi_t4_ex_vs5.reset_index()

## copy to new data frame
aesi_t4_ex_vs7 = aesi_t4_ex_vs6.copy() 

## Set up initial variables
_usubjid = aesi_t4_ex_vs7.loc[0,'USUBJID'] # first usubjid
_aedecod = aesi_t4_ex_vs7.loc[0,'AEDECOD'] # first usubjid
_aestdtc = aesi_t4_ex_vs7.loc[0,'AESTDTC'] # first AE start date
_vstestcd = aesi_t4_ex_vs7.loc[0, 'VSTESTCD'] # first VS test code
last_index = aesi_t4_ex_vs7.index[-1] ## last index number 

## use for-loop to derive last day of exposure
for i, _df11 in aesi_t4_ex_vs7.iterrows():
    
    #Last observation of each group - USUBJID & AEDECOD & AESTDTC 
    if i > 0:
        if _df11['USUBJID'] != _usubjid:
            aesi_t4_ex_vs7.loc[i-1, 'LAST_OBS'] = 1
            _usubjid = _df11['USUBJID']
        if _df11['AEDECOD'] != _aedecod:
            aesi_t4_ex_vs7.loc[i-1, 'LAST_OBS'] = 1
            _aedecod = _df11['AEDECOD']
        if _df11['AESTDTC'] != _aestdtc:
            aesi_t4_ex_vs7.loc[i-1, 'LAST_OBS'] = 1
            _aestdtc = _df11['AESTDTC']
        if _df11['VSTESTCD'] != _vstestcd:
            aesi_t4_ex_vs7.loc[i-1, 'LAST_OBS'] = 1
            _vstestcd = _df11['VSTESTCD']    
            
    if i == (last_index):  # for last record  
        aesi_t4_ex_vs7.loc[i, 'LAST_OBS'] = 1

    # Derive Days of exposures from AE date
    if (_df11.AESTDTC > _df11.EXENDTC) and (len(_df11.EXENDTC ) == 10 ) and (len(_df11.EXSTDTC ) == 10 ):
        aesi_t4_ex_vs7.loc[i, 'AE_EXSTDY'] = (dt.datetime.strptime(_df11.EXENDTC, '%Y-%m-%d') - 
                                                 dt.datetime.strptime(_df11.EXSTDTC, '%Y-%m-%d')).days + 1 
    elif (_df11.AESTDTC <= _df11.EXENDTC) and (len(_df11.EXENDTC ) == 10 ) and (len(_df11.EXSTDTC ) == 10 ):
        aesi_t4_ex_vs7.loc[i, 'AE_EXSTDY'] = (dt.datetime.strptime(_df11.AESTDTC, '%Y-%m-%d') - 
                                                dt.datetime.strptime(_df11.EXSTDTC, '%Y-%m-%d')).days 

    # Derive Days of exposure from Vital date
    if (_df11.VSDTC > _df11.EXENDTC) and (len(_df11.EXENDTC ) == 10 ) and (len(_df11.EXSTDTC ) == 10 ) and (len(_df11.VSDTC ) == 10 ):
        aesi_t4_ex_vs7.loc[i, 'VS_EXSTDY'] = (dt.datetime.strptime(_df11.EXENDTC, '%Y-%m-%d') - 
                                                 dt.datetime.strptime(_df11.EXSTDTC, '%Y-%m-%d')).days + 1 
    elif (_df11.VSDTC <= _df11.EXENDTC) and (len(_df11.EXENDTC ) == 10 ) and (len(_df11.EXSTDTC ) == 10 ) and (len(_df11.VSDTC ) == 10 ):
        aesi_t4_ex_vs7.loc[i, 'VS_EXSTDY'] = (dt.datetime.strptime(_df11.VSDTC, '%Y-%m-%d') - 
                                                dt.datetime.strptime(_df11.EXSTDTC, '%Y-%m-%d')).days 
        
## Kepp the last vitals before AESI date 
aesi_t4_ex_vs8 = aesi_t4_ex_vs7[aesi_t4_ex_vs7.LAST_OBS == 1]

## reset index 
aesi_t4_ex_vs9 = aesi_t4_ex_vs8.reset_index()

print(aesi_t4_ex_vs9.columns.tolist())
aesi_t4_ex_vs = aesi_t4_ex_vs9.drop(columns=['LAST_OBS','index','level_0'])
print(len(aesi_t4_ex_vs))
aesi_t4_ex_vs[['USUBJID','AESTDTC','VSDTC','EXSTDTC','EXENDTC','AE_EXSTDY','VS_EXSTDY']]

['level_0', 'index', 'STUDYID', 'USUBJID', 'SEX', 'AGE', 'AETERM', 'AEDECOD', 'AESTDTC', 'AEACN', 'AESER', 'AESEV', 'WEIGHTB', 'AEOUT', 'EXDOSTXT', 'EXSTDTC', 'EXENDTC', 'VSTESTCD', 'VISIT', 'VISITNUM', 'VSDTC', 'VSSTRESC', 'VSSTRESU', 'AE_EXSTDY', 'LAST_OBS', 'VS_EXSTDY']
21


,USUBJID,AESTDTC,VSDTC,EXSTDTC,EXENDTC,AE_EXSTDY,VS_EXSTDY
0,ZX008-1501-0101-01,2017-01-04,,2016-11-29,2017-06-22,36.0,NaN
1,ZX008-1501-0103-03,2017-08-08,2017-08-08,2017-02-09,2018-02-04,180.0,180.0
2,ZX008-1501-0103-12,2017-08-09,2017-08-09,2017-04-14,2018-06-18,117.0,117.0
3,ZX008-1501-0103-12,2018-02-06,2018-02-06,2017-04-14,2018-06-18,298.0,298.0
4,ZX008-1501-0103-12,2017-08-09,2017-08-09,2017-04-14,2018-06-18,117.0,117.0
5,ZX008-1501-0103-16,2017-07-27,2017-07-27,2017-06-22,2018-02-21,35.0,35.0
6,ZX008-1501-0103-16,2017-07-27,2017-07-27,2017-06-22,2018-02-21,35.0,35.0
7,ZX008-1501-0110-11,2018-02-19,2018-02-19,2017-10-16,2018-02-18,126.0,126.0
8,ZX008-1501-0115-03,2018-03-07,,2017-12-20,2018-05-28,77.0,NaN
9,ZX008-1501-0119-09,2018-02-15,2018-02-15,2018-01-17,2018-02-14,29.0,29.0


In [322]:
## Final data set for table 4
aesi_t4_final = aesi_t4_ex_vs

In [323]:
print(aesi_t3_final.columns.tolist())

['STUDYID', 'USUBJID', 'SEX', 'AGE', 'WEIGHTB', 'AETERM', 'AEDECOD', 'AESTDTC', 'AEACN', 'AESER', 'AESEV', 'AEOUT', 'LBTESTCD', 'LBSTRESC', 'LBSTRESU', 'LBSTNRLO', 'LBSTNRHI', 'LBDTC', 'VISIT', 'LBDT', 'EXDOSTXT', 'EXSTDTC', 'EXENDTC', 'XSTESTCD', 'XSTEST', 'XSORRES', 'XSDTC', 'XSSTDTC', 'DAY_XS_LB', 'LB_EXSTDY']


In [326]:
## print final datasets to local drive

f5 = open('./out/1503-CSR-aesi_final.rtf', 'w')
print("1503 CSR AESI Reports\n\n", file=f5)

print("Table 2: Listing of Subjects with Trace Mitral and/or Aortic Regurgitation", file=f5)
print(aesi_t2_final[['STUDYID','USUBJID','SEX','AGE','WEIGHTB','AETERM','AEDECOD','AESTDTC','AEACN','AESER']].to_string(), 
      file=f5)
print("\n", file=f5)
print(aesi_t2_final[['AESTDTC','ECGTEST', 'EGDTC','EXDOSTXT','EXSTDTC','EXENDTC','0.2mg','0.4mg','0.6mg','EG_EXSTDY',
                           'ECHO_SUB2']].to_string(), file=f5)
print("\n\n", file=f5)

print("Table 3: Listing of Subjects with Blood Prolactin Increased", file=f5)
print(aesi_t3_lb_ex_xs[['STUDYID', 'USUBJID', 'SEX', 'AGE','WEIGHTB', 'AEDECOD', 'AESTDTC', 
                        'AESEV', 'AEACN', 'AESER',  'AEOUT']].to_string(), file=f5)
print("\n", file=f5)
print(aesi_t3_lb_ex_xs[['LBTESTCD', 'LBSTRESC', 'LBSTRESU', 'LBSTNRLO', 'LBSTNRHI','VISIT', 'LBDT','EXDOSTXT', 
                        'EXSTDTC', 'EXENDTC','XSTEST', 'XSSTDTC', 'DAY_XS_LB','LB_EXSTDY']].to_string(), file=f5)
print("\n\n", file=f5)

print("Table 4: Listing of Subjects with Blood Pressure, Heart Rate Increased", file=f5)
print(aesi_t4_final[['STUDYID', 'USUBJID', 'SEX', 'AGE', 'AEDECOD', 'AESTDTC', 
                        'AEACN', 'AESER', 'WEIGHTB', 'AEOUT','AESEV']].to_string(), file=f5)
print("\n", file=f5)
print(aesi_t4_final[['EXDOSTXT', 'EXSTDTC', 'EXENDTC', 'VSTESTCD', 'VISIT', 
                        'VSDTC', 'VSSTRESC', 'VSSTRESU','AE_EXSTDY','VS_EXSTDY']].to_string(), file=f5)
f5.close()

